# Data Engineering

A NOTE: Each of these sections was a separate file which I merged into this notebook per instructions. That's why each section is treated as a separate notebook.

## Scraping oglasnik.com

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as req


I'm first trying to get as many cars as possible from different ads. The only filter I'm implementing at the beginning is that
a) the car is used,
b) that it has the full price listed,
c) is not sold for parts only
as I want to look at the entire used cars market in Croatia. First I'm scraping used car ads from oglasnik.hr.

In [5]:
lastPageScraped = None
dataOglasnik = []

In [6]:
baseUrlOglasnik = "https://www.oglasnik.hr/prodaja-automobila?page="

for i in range(1, 206):
    pageData = []
    
    if (lastPageScraped is None) or (lastPageScraped < i):
        print("XXXXXXXXXXXXXXXXXXX page ", i, "of 205")
        urlOglasnik = baseUrlOglasnik + str(i)
        response = req.get(urlOglasnik)

        soup = BeautifulSoup(response.content, "html.parser")

        # gets all the anchor elements on a page and all the html inside them
        linksOnAPage = soup.find_all("a", class_="izdvojeno-osnovni")

        #iterating through the anchors to get to each car
        for link in linksOnAPage:
            adUrlOglasnik = link.get("href")
            print(adUrlOglasnik)

            #now i'm opening the link to get to the page i want to scrape because it has all the detail that the ad doesn't have
            responseAd = req.get(adUrlOglasnik)
            soupAd = BeautifulSoup(responseAd.content, "html.parser")
            topDetails = soupAd.find("div", class_ = "top-details")

            p_tagLocation = topDetails.find('p')
            location = p_tagLocation.contents[1] if p_tagLocation.contents[1] else ""

            span_tagActiveDate = topDetails.find('span', recursive = False)
            b_tagsActiveDate = span_tagActiveDate.find_all('b')
            activeSince = b_tagsActiveDate[0].contents[0][3:]

            timesViewed = b_tagsActiveDate[1].contents[0].split()[0]

            dl_tagPrice = topDetails.find("dl", class_="price")
            if dl_tagPrice:
                price = dl_tagPrice.find("dd").contents[0].split()[0]
            else:
                price = ""

            #now i have everything that i need from this part of the page, and i'm saving it in a dictionary that i can later
            #add to from the bottom parts of the page
            instance = {
                'Location': location,
                'Active_Since': activeSince,
                'Times_Viewed': timesViewed,
                'Price_in_Euros': price
            }

            oglasDetails = soupAd.find("div", class_ = "oglas-details")
            classifiedDetails = oglasDetails.find_all("div", class_ = "classified-details")

            #Now, for this section where more about cars is displayed, each ad has a unique combination of info provided by the seller
            #Some cars have a lot of very detailed info and some have almost none apart from maybe brand and model.
            #This part of the code inside this loop extracts everything from "Podaci o vozilu" and "Dodatni podaci" sections
            #without me knowing what piece of info it is and it saves it into the dictionary defined above as a key: value pair.
            #I decided to do it like this here and then decide which categories of info I can use and which not later on and clean it
            #all up at a later stage.
            for section in classifiedDetails:
                strong_tag = section.find("strong", class_ = "tab-header")
                if strong_tag is None:
                    continue

                sectionTitle = strong_tag.contents[0] 
                if (sectionTitle != "Podaci o vozilu") & (sectionTitle != "Dodatni podaci"):
                    continue                

                span_tags = section.find_all("span", class_="color-light")
                for span_tag in span_tags:
                    key = span_tag.contents[0].replace(" ", "_").replace(":", "")
                    value = span_tag.next_sibling.strip()
                    instance[key] = value

            pageData.append(instance) 
        dataOglasnik = dataOglasnik + pageData
        lastPageScraped = i

XXXXXXXXXXXXXXXXXXX page  1 of 205
https://www.oglasnik.hr/prodaja-automobila/golf-4-1-6i-lpg-climatronik-oglas-5237526
https://www.oglasnik.hr/prodaja-automobila/citroen-c4-grand-picasso-2-0-hdi-automatik-7-sjedala-garancija-12mj-oglas-5199516
https://www.oglasnik.hr/prodaja-automobila/skoda-kodiaq-2-0tdi-dsg-style-virtual-kokpit-full-led-ambi-rasvjeta-oglas-5210218
https://www.oglasnik.hr/prodaja-automobila/vw-tiguan-2-0-tdi-dsg-full-led-grijanje-sjedala-parking-kamera-oglas-5210222
https://www.oglasnik.hr/prodaja-automobila/dacia-logan-mcv-oglas-5224444
https://www.oglasnik.hr/prodaja-automobila/skoda-octavia-1-6-tdi-oglas-5237490
https://www.oglasnik.hr/prodaja-automobila/bmw-serije-2-active-tourer-diesel-izvrstan-i-ocuvan-oglas-5237054
https://www.oglasnik.hr/prodaja-automobila/seat-ibiza-1-2-12v-1-vlasnik-garazirana-kao-nova-oglas-5237479
https://www.oglasnik.hr/prodaja-automobila/outlander-2-4-benzin-plin-2006-g-oglas-4732283
https://www.oglasnik.hr/prodaja-automobila/vw-caddy-2

https://www.oglasnik.hr/prodaja-automobila/mercedes-e-klasa-amg-e53-4matic-oglas-5237003
https://www.oglasnik.hr/prodaja-automobila/jaguar-f-pace-3-0d-awd-oglas-5237001
https://www.oglasnik.hr/prodaja-automobila/bmw-serija-7-750d-xdrive-aut-oglas-5237000
https://www.oglasnik.hr/prodaja-automobila/bmw-serija-3-320d-m-sport-oglas-5236999
https://www.oglasnik.hr/prodaja-automobila/land-rover-range-rover-evoque-r-dynamic-se-2-0d-td4-mhev-204hp-awd-a9-oglas-5236998
https://www.oglasnik.hr/prodaja-automobila/land-rover-discovery-sport-se-2-0-204-mhev-awd-a9-oglas-5236997
https://www.oglasnik.hr/prodaja-automobila/land-rover-discovery-sport-r-dyn-se-2-0-163-mhev-awd-a9-oglas-5236996
https://www.oglasnik.hr/prodaja-automobila/jaguar-f-pace-r-dyn-se-2-0-204-awd-a8-mhev-oglas-5236995
https://www.oglasnik.hr/prodaja-automobila/jaguar-f-pace-r-dyn-se-2-0-204-awd-a8-mhev-oglas-5236994
https://www.oglasnik.hr/prodaja-automobila/jaguar-f-pace-r-dyn-se-2-0-204-awd-a8-mhev-oglas-5236993
https://www.ogl

https://www.oglasnik.hr/prodaja-automobila/land-rover-discovery-sport-hse-d4-150-km-4wd-ugodno-kot-nov-oglas-5236381
https://www.oglasnik.hr/prodaja-automobila/mercedes-a-klasa-1-8-cdi-2007-godina-oglas-5236364
XXXXXXXXXXXXXXXXXXX page  10 of 205
https://www.oglasnik.hr/prodaja-automobila/golf-v-oglas-5232112
https://www.oglasnik.hr/prodaja-automobila/mercedes-b200-cdi-dizel-automatic-2011-g-oglas-5199071
https://www.oglasnik.hr/prodaja-automobila/golf-5-variant-oglas-5236311
https://www.oglasnik.hr/prodaja-automobila/ford-c-max-oglas-5236337
https://www.oglasnik.hr/prodaja-automobila/renault-clio-limited-energy-dci-75-oglas-5236344
https://www.oglasnik.hr/prodaja-automobila/land-rover-range-rover-evoque-r-dyn-se-2-0-mhev-163-awd-a9-oglas-5236343
https://www.oglasnik.hr/prodaja-automobila/land-rover-discovery-sport-r-dyn-se-2-0-163-mhev-awd-a9-oglas-5236341
https://www.oglasnik.hr/prodaja-automobila/land-rover-discovery-sport-r-dyn-se-2-0-204-mhev-awd-a9-oglas-5236342
https://www.oglas

https://www.oglasnik.hr/prodaja-automobila/peugeot-2008-2018-god-1-6-hdi-100-ks-66-000km-reg-03-24-oglas-5236114
https://www.oglasnik.hr/prodaja-automobila/audi-a4-karavan-oglas-5233805
https://www.oglasnik.hr/prodaja-automobila/passat-2-3-5v-oglas-5233801
https://www.oglasnik.hr/prodaja-automobila/mercedes-benz-115-220d-prilika-za-ljubitelje-oglas-5233787
https://www.oglasnik.hr/prodaja-automobila/dacia-logan-2020-oglas-5233775
https://www.oglasnik.hr/prodaja-automobila/a6-v6-2-8-quatrro-oglas-5233756
https://www.oglasnik.hr/prodaja-automobila/opel-vivaro-1-9-cdti-oglas-5233748
https://www.oglasnik.hr/prodaja-automobila/bmw-serija-3-320d-aut-klima-tempomat-alu-putno-racunalo-odlican-oglas-5233742
https://www.oglasnik.hr/prodaja-automobila/mazda-3-1-4i-16v-1-vlasnik-1-boja-oglas-5233740
https://www.oglasnik.hr/prodaja-automobila/mazda-mazda2-sport-1-3i-oglas-5233737
https://www.oglasnik.hr/prodaja-automobila/vw-passat-variant-1-6-tdi-bmt-comfortline-oglas-5233736
https://www.oglasnik.h

https://www.oglasnik.hr/prodaja-automobila/smart-fortwo-coupe-electric-12-900-00-oglas-5233020
https://www.oglasnik.hr/prodaja-automobila/audi-a8-4-2-tdi-v8t-quattro-33-000-00-oglas-5233018
https://www.oglasnik.hr/prodaja-automobila/skoda-octavia-combi-1-6-tdi-greente-style-13-100-00-oglas-5233016
https://www.oglasnik.hr/prodaja-automobila/mercedes-3-0-gle-350-d-4matic-jamstvo-15-mjeseci-49-900-00-oglas-5233017
https://www.oglasnik.hr/prodaja-automobila/peugeot-308-1-5-hdi-active-jamstvo-15-mjeseci-17-640-00-oglas-5233015
https://www.oglasnik.hr/prodaja-automobila/volvo-s80-2-0d4-interception-jamstvo-15-mjeseci-20-900-00-oglas-5233014
https://www.oglasnik.hr/prodaja-automobila/audi-q5-55tfsie-quattro-s-tr-s-line-83-900-00-oglas-5233013
https://www.oglasnik.hr/prodaja-automobila/opel-grandland-x-2-0-cdti-aut-ultimate-jamstvo-15-mjeseci-21-900-00-oglas-5233012
https://www.oglasnik.hr/prodaja-automobila/renault-megane-1-5-dci-intens-automatic-12-300-00-oglas-5233011
https://www.oglasnik.h

...

https://www.oglasnik.hr/prodaja-automobila/opel-astra-k-st-oglas-5099147
https://www.oglasnik.hr/prodaja-automobila/audi-a1-30-tfsi-sline-oglas-5099138
https://www.oglasnik.hr/prodaja-automobila/vw-golf-vii-1-6-tdi-trendline-oglas-5099136
https://www.oglasnik.hr/prodaja-automobila/mercedes-benz-e-coupe-220d-automatik-amg-panorama-burmester-kamera-oglas-5098359
https://www.oglasnik.hr/prodaja-automobila/mercedes-benz-a-klasa-a180d-navigacija-kamera-oglas-5098355
https://www.oglasnik.hr/prodaja-automobila/mercedes-benz-c-klasa-t-model-200d-amg-automatik-navi-led-kamera-oglas-5098354
https://www.oglasnik.hr/prodaja-automobila/nissan-qashqai-1-6-dci-aut-96kw-5-godina-garancije-oglas-5098351
https://www.oglasnik.hr/prodaja-automobila/vw-passat-2-0-tdi-dsg-4m-highline-oglas-5098347
https://www.oglasnik.hr/prodaja-automobila/audi-a6-3-0-tdi-quattro-avant-oglas-5098346
https://www.oglasnik.hr/prodaja-automobila/vw-touran-1-6-tdi-dsg-oglas-5097778
https://www.oglasnik.hr/prodaja-automobila/citr

https://www.oglasnik.hr/prodaja-automobila/opel-astra-1-6-cdti-enjoy-oglas-5076160
https://www.oglasnik.hr/prodaja-automobila/opel-corsa-1-4-colour-edition-oglas-5076161
https://www.oglasnik.hr/prodaja-automobila/audi-a6-3-0-tdi-quattro-s-tronic-oglas-5076158
https://www.oglasnik.hr/prodaja-automobila/renault-kadjar-dci-110-energy-limited-edc-automatic-oglas-5076156
https://www.oglasnik.hr/prodaja-automobila/renault-clio-dci-75-limited-navi-alu-led-reg-do-04-2023-oglas-5076154
https://www.oglasnik.hr/prodaja-automobila/skoda-fabia-karavan-1-0-tsi-70kw-5-godina-garancije-oglas-5076027
https://www.oglasnik.hr/prodaja-automobila/skoda-kodiaq-2-0-tdi-dsg-active-oglas-5074969
https://www.oglasnik.hr/prodaja-automobila/ford-focus-karavan-1-5-tdci-business-oglas-5074669
https://www.oglasnik.hr/prodaja-automobila/opel-insignia-edition-grandsport-1-6cdti-136-ks-oglas-5074583
https://www.oglasnik.hr/prodaja-automobila/bmw-6-grandcoupe-640xd-m-paket-oglas-5074580
https://www.oglasnik.hr/prodaja-a

https://www.oglasnik.hr/prodaja-automobila/skoda-octavia-2-0-slx-oglas-4999400
https://www.oglasnik.hr/prodaja-automobila/opel-insignia-sports-tourer-1-6-cdti-ecotec-edition-oglas-4999339
https://www.oglasnik.hr/prodaja-automobila/opel-insignia-karavan-1-6-cdti-oglas-4998997
https://www.oglasnik.hr/prodaja-automobila/skoda-fabia-1-4-tdi-ambition-oglas-4998981
https://www.oglasnik.hr/prodaja-automobila/bmw-x2-sdrive-18d-oglas-4998983
XXXXXXXXXXXXXXXXXXX page  198 of 205
https://www.oglasnik.hr/prodaja-automobila/vw-passat-2-0-tdi-oglas-4998978
https://www.oglasnik.hr/prodaja-automobila/volvo-v90-d3-momentum-oglas-4993346
https://www.oglasnik.hr/prodaja-automobila/mini-countryman-1-5-automatik-navi-kamera-panorama-oglas-4993094
https://www.oglasnik.hr/prodaja-automobila/skoda-octavia-2-0-tdi-dsg-sw-oglas-4993085
https://www.oglasnik.hr/prodaja-automobila/volvo-v40-d2-kinetic-oglas-4992704
https://www.oglasnik.hr/prodaja-automobila/renault-grand-scenic-1-5-dci-7-sjedala-oglas-4992401
http

https://www.oglasnik.hr/prodaja-automobila/audi-a4-2-0-tdi-s-tronic-nije-uvoz-oglas-4854591
https://www.oglasnik.hr/prodaja-automobila/audi-a3-sportback-2-0-tdi-s-tronic-navigacija-oglas-4846556
https://www.oglasnik.hr/prodaja-automobila/land-rover-range-rover-sport-4-4-v8-se-plin-2015g-oglas-4834690
https://www.oglasnik.hr/prodaja-automobila/renault-koleos-2-0dci-175-4wd-energy-initiale-paris-oglas-4830511
https://www.oglasnik.hr/prodaja-automobila/renault-koleos-2-0dci-175-4wd-energy-initiale-paris-oglas-4830510
https://www.oglasnik.hr/prodaja-automobila/insignia-st-edition-1-6-oglas-4824740
XXXXXXXXXXXXXXXXXXX page  202 of 205
https://www.oglasnik.hr/prodaja-automobila/dacia-logan-mcv-1-0-sce-75-ambiance-oglas-4818780
https://www.oglasnik.hr/prodaja-automobila/renault-clio-v-limited-tce-90-oglas-4818777
https://www.oglasnik.hr/prodaja-automobila/bmw-serija-1-118i-automatik-oglas-4811817
https://www.oglasnik.hr/prodaja-automobila/bmw-serija-1-116i-m-paket-navi-led-kamera-oglas-481181

https://www.oglasnik.hr/prodaja-automobila/opel-astra-sw-enjoy-1-6-cdti-oglas-4308337
https://www.oglasnik.hr/prodaja-automobila/jaguar-xe-pure-2-0d-rwd-oglas-4308334
https://www.oglasnik.hr/prodaja-automobila/land-rover-range-rover-sport-3-0d-oglas-4303272
https://www.oglasnik.hr/prodaja-automobila/opel-grandland-x-1-5-cdti-enjoy-oglas-4276209
https://www.oglasnik.hr/prodaja-automobila/mercedes-a-klasa-200d-oglas-4267418
https://www.oglasnik.hr/prodaja-automobila/opel-astra-at-hb-enjoy-1-6-cdti-oglas-4252519
https://www.oglasnik.hr/prodaja-automobila/opel-astra-sw-enjoy-1-6-cdti-oglas-4249060
https://www.oglasnik.hr/prodaja-automobila/opel-zafira-2-0-cdti-5-2-edition-oglas-4248998
https://www.oglasnik.hr/prodaja-automobila/jaguar-xe-2-0d-at-oglas-4238973
https://www.oglasnik.hr/prodaja-automobila/jaguar-f-pace-prestige-2-0d-180hp-awd-a8-oglas-4238971
https://www.oglasnik.hr/prodaja-automobila/insignia-grand-sport-edition-b1-6-dth-s-s-100kw-136ks-mt6-oglas-3904108


In [7]:
df = pd.DataFrame(dataOglasnik)
display(df)

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Pogon,Klima_uređaj,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer
0,"Sesvete, Grad Zagreb",29.03.2023.,4,1.000,VW,Golf IV,1998,370000 km,07/2023,74 kW,...,,,,,,,NaN,NaN,NaN,NaN
1,"Novi Zagreb - Istok, Grad Zagreb",29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015,128400 km,NaN,110 kW,...,,,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
2,"Novi Zagreb, Grad Zagreb",29.03.2023.,264,31.000,Škoda,Kodiaq,2019,63400 km,NaN,110 kW,...,,,NaN,NaN,NaN,,,NaN,NaN,NaN
3,"Novi Zagreb, Grad Zagreb",29.03.2023.,330,24.400,VW,Tiguan,2017,113600 km,NaN,110 kW,...,,,NaN,NaN,NaN,,,NaN,NaN,NaN
4,Grad Zagreb,29.03.2023.,196,5.500,Dacia,Logan,2015,257000 km,04/2023,66 kW,...,,,,,NaN,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,"Arsenal, Pula, Istarska",08.01.2021.,540,"11.533,61",NaN,NaN,2018,164118 km,NaN,81 kW,...,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
4093,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,402,17.400,NaN,NaN,2017,132210 km,NaN,125 kW,...,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
4094,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,682,21.900,NaN,NaN,2017,92908 km,NaN,110 kW,...,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN
4095,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,634,39.500,NaN,NaN,2017,96524 km,NaN,132 kW,...,,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN


In [8]:
df.describe()

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Pogon,Klima_uređaj,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer
count,4097,4097,4097,4097,3056,2839,4092,3223,989,3598,...,2351,1955,1672,1172,617,3063,768,798,412,30
unique,244,265,759,1400,50,411,89,2055,41,203,...,1,1,1,1,1,1,1,1,1,1
top,[],09.01.2023.,276,1,VW,Passat,2018,1 km,03/2024,110 kW,...,,,,,,,,,,
freq,618,298,51,117,483,81,519,34,93,341,...,2351,1955,1672,1172,617,3063,768,798,412,30


Before I clean this data, I'll go and scrape the other websites that I want to and then do the cleaning when I know exactly what columns I have on hand.

In [9]:
df.to_csv('rawOglasnik.csv', index = False)

## Scraping index.com

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as req

In [2]:
lastPageScraped = None
dataIndex = []

In [6]:
def scrape(start, end):
    baseUrlIndex = "https://www.index.hr/oglasi/osobni-automobili/gid/27?pojamZup=-2&tipoglasa=1&sortby=1&grad=0&naselje=0&cijenaod=1000&cijenado=19500000&attr_bit_349=1&attr_bit_350=1&attr_bit_351=1&vezani_na=179-1190_470-910_1172-1335_359-1192&elementsNum=50&num="
    global lastPageScraped
    global dataIndex
    
    for i in range(start, end):
        pageData = []
        #This is a simple system to save everything I scraped up until the first error, in case errors happen while I'm still 
        #developing the code.
        if (lastPageScraped is None) or (lastPageScraped < i):
            print("xxxxxxxxxxxxxxxxxxxxx page ", i, "of ")
            urlIndex = baseUrlIndex + str(i)
            response = req.get(urlIndex)

            soup = BeautifulSoup(response.content, "html.parser")

            # gets all the anchor elements on a page and all the html inside them
            linksOnAPage = soup.find_all("a", class_="result")


            #this loop is looping through one car at the time
            for link in linksOnAPage:
                try:
                    adUrlIndex = link.get("href")
                    #if adUrlIndex == "https://www.index.hr/oglasi/mercedes-benz-c-klasa-220d-bluetec-servisna-knjiga-reg-01-2024god/oid/3946952":
                        #print(nonsensesoitthrows)
                    print(adUrlIndex)

                    #now i'm opening the link to get to the page i want to scrape because it has all the detail that the ad doesn't have
                    responseAd = req.get(adUrlIndex)
                    soupAd = BeautifulSoup(responseAd.content, "html.parser")

                    price_tag = soupAd.find("div", class_="price")
                    if price_tag:
                        price = price_tag.find("span").contents[0].split()[0]
                    else:
                        continue

                    location_li= soupAd.find("div", class_="oglasivac_info").find("ul").find_all("li")[3]
                    location = location_li.contents[0].split(",")[0].strip()

                    publishedInfo = soupAd.find("div", class_="published").contents[2].split(":")
                    activeSince = publishedInfo[1].split()[0]
                    timesViewed = publishedInfo[3].split()[0]

                    #i figured that a lot of cars are missing the brand and model in the info list, so i'm saving the title in hopes of being
                    #able to get to that info from the title so i don't have to drop that row just because i don't have the brand and model
                    #and that's the info i really need
                    title = soupAd.find("div", class_="likeH1").contents[0]

                    instance = {
                        'Location': location,
                        'Active_Since': activeSince,
                        'Times_Viewed': timesViewed,
                        'Price_in_Euros': price,
                        '_Title': title
                    }

                    oglasHolders = soupAd.find_all("div", class_="oglasHolder_1")

                    #this outer part of the loop accesses each of two different sections with more detail about the vehicle
                    for section in oglasHolders:
                        featureWrappers = section.find_all("div", class_="features-wrapper")

                        #each of those sections is for some reason split into left and right hand side, so that's the reason for this loop
                        for wrapper in featureWrappers:
                            #each wrapper has a certain amount of rows, and this loops through them
                            infoRows = wrapper.find_all("ul")

                            #final loop which is accessing rows one by one and adding the info as key: value pairs into the dictionary
                            #that was defined before
                            for infoRow in infoRows:
                                li_tags = infoRow.find_all("li")

                                key = li_tags[0].contents[0].strip().replace(" ", "_").replace(":", "")
                                value = li_tags[1].contents[0].strip()

                                instance[key] = value   
                    pageData.append(instance) 
                except:
                    continue

            dataIndex = dataIndex + pageData
            lastPageScraped = i
        
        
        
        
        
        

In [3]:
print(len(dataIndex))

0


In [4]:
print(lastPageScraped)

None


In [ ]:
scrape(203, 253)
df1 = pd.DataFrame(dataIndex)

In [14]:
df1.to_csv("rawIndex203-253savepoint.csv", index = False)

In [ ]:
scrape(253, 303)
df2 = pd.DataFrame(dataIndex)
df2.to_csv("rawIndex203-303savepoint.csv", index = False)

The previous cells' output is missing because I accidentally ran the code and it threw an error. It looked the same as for all the other pages.

In [8]:
scrape(303, 353)
df3 = pd.DataFrame(dataIndex)
df3.to_csv("rawIndex303-353savepoint.csv", index = False)

xxxxxxxxxxxxxxxxxxxxx page  303 of 
https://www.index.hr/oglasi/opel-astra-j-st-karavan-1-7-cdti-innovation/oid/3692624
https://www.index.hr/oglasi/audi-a6-avant-3-0-tdi-quattro-2017/oid/3692601
https://www.index.hr/oglasi/mercedes-benz-c-klasa-220-cdi-amg/oid/3692594
https://www.index.hr/oglasi/toyota-aygo-1-0-vvt-i/oid/3692547
https://www.index.hr/oglasi/prodajem-opel-corsu-1-3-cdti-klima/oid/3692484
https://www.index.hr/oglasi/jeep-cherokee-2-8-sport-td/oid/3692479
https://www.index.hr/oglasi/ford-focus-1-6-lpg/oid/3692410
https://www.index.hr/oglasi/mercedes-benz-e-klasa-coupe-panorama-led/oid/3692398
https://www.index.hr/oglasi/renault-scenic-1-5-dci-autenthique/oid/3692396
https://www.index.hr/oglasi/renault-clio-1-5-dci-community/oid/3692379
https://www.index.hr/oglasi/opel-vectra-2-0-dti/oid/3692374
https://www.index.hr/oglasi/bmw-serija-3-318d-2-0/oid/3692323
https://www.index.hr/oglasi/vw-passat-1-6-tdi/oid/3692314
https://www.index.hr/oglasi/audi-a6-2-0-tdi/oid/3439900
https

https://www.index.hr/oglasi/renault-megane-grandtour-dci-130/oid/3689048
https://www.index.hr/oglasi/peugeot-308-sw-1-6-bluehdi/oid/3689045
https://www.index.hr/oglasi/mercedes-e-klasa-200d-velika-navigacija-kamera-avantgarde/oid/3689005
https://www.index.hr/oglasi/fiat-500-sport-s-s-twin-air-turbo-2012/oid/3688978
https://www.index.hr/oglasi/vw-golf-vii-top-stanje-kupljen-nov-u-hr/oid/3688954
https://www.index.hr/oglasi/opel-zafira-tourer-cosmo/oid/3688951
https://www.index.hr/oglasi/mini-countryman-10-2015g-diesel-47-000-km/oid/3688949
https://www.index.hr/oglasi/kia-pro-ceed-1-6-benzin/oid/3688932
https://www.index.hr/oglasi/audi-a3-sedan-2-0-tdi-150-ks-aut-navi-extra-set-felgi-bang-olufsen-sms-poziv-091-2413-227/oid/3688922
https://www.index.hr/oglasi/vw-golf-vi-2-0-tdi/oid/3688914
https://www.index.hr/oglasi/audi-a4-avant-2-0/oid/3688865
https://www.index.hr/oglasi/lexus-ct-200-h/oid/3688821
https://www.index.hr/oglasi/kia-sorento-2-2-crdi-ex-platinum-awd/oid/3656814
https://www.i

https://www.index.hr/oglasi/vw-passat-variant-1-6-tdi-bmt/oid/3683974
https://www.index.hr/oglasi/mercedes-benz-a-klasa-sport/oid/3683941
https://www.index.hr/oglasi/bmw-e38-740i-v8-286-ks/oid/3683923
https://www.index.hr/oglasi/mazda-6-2-5-gta/oid/3683855
https://www.index.hr/oglasi/vw-passat-2-0-tdi/oid/3683854
https://www.index.hr/oglasi/bmw-serija-3-touring-318d-e91-lci/oid/3683791
https://www.index.hr/oglasi/bmw-serija-3-compact-320td/oid/3683752
https://www.index.hr/oglasi/bmw-serija-7-740xd/oid/3683744
https://www.index.hr/oglasi/chevrolet-kalos-hatchback-1-2-se-53kw-2004-god/oid/3683742
https://www.index.hr/oglasi/mercedes-benz-a-klasa-180-cdi/oid/3683729
https://www.index.hr/oglasi/mercedes-benz-e-klasa-w210-200-cdi/oid/3683710
https://www.index.hr/oglasi/bmw-serija-3-318-e-46/oid/3683691
https://www.index.hr/oglasi/audi-a1-1-6-tdi-registriran-godinu-dana/oid/3683637
https://www.index.hr/oglasi/opel-insignia-karavan-20cdti/oid/3683591
xxxxxxxxxxxxxxxxxxxxx page  308 of 
https:

https://www.index.hr/oglasi/ssang-yong-musso-2-9-td-pick-up-4x4/oid/3680996
https://www.index.hr/oglasi/peugeot-308-limuzina/oid/3680995
https://www.index.hr/oglasi/bmw-serija-1-2-0-diesel/oid/3680958
https://www.index.hr/oglasi/renault-clio-iii/oid/3680952
https://www.index.hr/oglasi/audi-a1-1-6-tdi-s-line/oid/3680930
https://www.index.hr/oglasi/vw-polo-1-6-tdi/oid/3680876
https://www.index.hr/oglasi/renault-espace/oid/3680875
https://www.index.hr/oglasi/vw-golf-vii-gtd/oid/3680873
https://www.index.hr/oglasi/peugeot-307-307-2-0-hdi/oid/3680861
https://www.index.hr/oglasi/audi-a4-2-0-tdi/oid/3680820
https://www.index.hr/oglasi/citroen-c5-2-0-hdi-automatik-regan-6-23/oid/3680809
https://www.index.hr/oglasi/bmw-serija-3-touring-320/oid/3680801
https://www.index.hr/oglasi/peugeot-207-super-stanje/oid/3680739
https://www.index.hr/oglasi/bmw-serija-3-touring-318d-automatik-10500e/oid/3680726
https://www.index.hr/oglasi/vw-golf-sportsvan-1-6-tdi-2015god/oid/3680721
https://www.index.hr/ogla

https://www.index.hr/oglasi/bmw-serija-1-118d/oid/3676470
https://www.index.hr/oglasi/peugeot-407-407-sw-2-0-hdi/oid/3676453
https://www.index.hr/oglasi/ford-c-max-1-6-hdi/oid/3676452
https://www.index.hr/oglasi/audi-q3-35-tfsi-s-tronic-s-line-black-optic/oid/3676422
https://www.index.hr/oglasi/vw-golf-v-2-0-tdi/oid/3675756
https://www.index.hr/oglasi/audi-q3-2-0-tdi/oid/3675732
https://www.index.hr/oglasi/bmw-serija-5-530-d-m-paket/oid/3675664
https://www.index.hr/oglasi/fiat-punto-1-2/oid/3675601
https://www.index.hr/oglasi/volvo-xc60-2-0d-d3-automatik-kupljen-u-rh/oid/3675594
https://www.index.hr/oglasi/hyundai-i30-karambol/oid/3675568
https://www.index.hr/oglasi/bmw-serija-1-118d-top-ponuda/oid/3675566
https://www.index.hr/oglasi/bmw-serija-1-118d/oid/3675557
https://www.index.hr/oglasi/bmw-serija-3-bmw-320-d/oid/3675524
https://www.index.hr/oglasi/peugeot-5008-automatik-1-6-bluehdi-85kw-s-s-eat6-allure-5d-7pl-servisiran-u-ovlastenom-servisu/oid/3675462
https://www.index.hr/oglasi/

...

https://www.index.hr/oglasi/audi-a4-avant-1-9-tdi-96-kw/oid/3618469
https://www.index.hr/oglasi/honda-cr-v-2-2-cdti-full-oprema/oid/3617923
https://www.index.hr/oglasi/kia-sportage-1-7-crdi-prvi-vlasnik/oid/3617912
https://www.index.hr/oglasi/citroen-berlingo-citroen-berlingo-2-0-hdi-xtr/oid/3617898
https://www.index.hr/oglasi/skoda-octavia-combi-4x4-top-stanje/oid/3617881
https://www.index.hr/oglasi/peugeot-406-coupe-2-0-benzin/oid/3617869
https://www.index.hr/oglasi/dacia-sandero-1-5-dci-sa-pdv-om/oid/3617807
https://www.index.hr/oglasi/mercedes-benz-a-klasa-150/oid/3617791
https://www.index.hr/oglasi/renault-4-1-1/oid/3617784
https://www.index.hr/oglasi/audi-a4-avant-2-0-tdi-190ks-s-line/oid/3617772
https://www.index.hr/oglasi/mercedes-benz-a-klasa-180-2-0-cdi/oid/3617768
xxxxxxxxxxxxxxxxxxxxx page  343 of 
https://www.index.hr/oglasi/citroen-c4-1-4-16v-lpg/oid/3617760
https://www.index.hr/oglasi/renault-megane-1-5-dci/oid/3617744
https://www.index.hr/oglasi/audi-a6-2-7tdi/oid/36177

https://www.index.hr/oglasi/chevrolet-cruze-1-6-14v-ls-lpg/oid/3614426
https://www.index.hr/oglasi/audi-a8l-4-0-tdi-v8/oid/3080197
https://www.index.hr/oglasi/bmw-serija-3-coupe-318is-323i/oid/3614406
https://www.index.hr/oglasi/ford-mondeo-2-0/oid/3614399
https://www.index.hr/oglasi/skoda-superb-2-0-tdi-pd-elegance/oid/3614398
https://www.index.hr/oglasi/mercedes-benz-c-klasa-w204-avantgarde-cash-12000eura/oid/3614392
https://www.index.hr/oglasi/mercedes-benz-c-klasa-200-d-amg-line/oid/3614391
https://www.index.hr/oglasi/alfa-romeo-159-sw-2-4-jtdm/oid/3614385
https://www.index.hr/oglasi/alfa-romeo-145-1-7-16v/oid/3614299
https://www.index.hr/oglasi/mercedes-benz-cls-klasa-350-4matic-amg/oid/3614294
https://www.index.hr/oglasi/renault-megane-scenic-1-6-16v-plin/oid/3614243
https://www.index.hr/oglasi/opel-astra-karavan-1-7cdti/oid/3614234
https://www.index.hr/oglasi/vw-passat-variant-1-6-tdi-confort/oid/3614220
https://www.index.hr/oglasi/bmw-serija-4-gran-coupe-435d-xdrive-hk-sag/oid/

https://www.index.hr/oglasi/audi-q3-sq3/oid/3609799
https://www.index.hr/oglasi/vw-caddy-1-6-tdi-n1/oid/3609789
https://www.index.hr/oglasi/renault-captur-90tce/oid/3609785
https://www.index.hr/oglasi/audi-a4-2-0tdi-100kw/oid/3609768
https://www.index.hr/oglasi/mg-zr-160-vvc/oid/3609753
https://www.index.hr/oglasi/mercedes-benz-r-klasa-320-cdi-4matic/oid/3609713
https://www.index.hr/oglasi/bmw-x5-bmw-x5-xdrive30d-jedinstven-s-tom-opremom-nema-na-trzistu/oid/3609709
https://www.index.hr/oglasi/audi-a4-2-0-tdi-automatic/oid/3609706
https://www.index.hr/oglasi/peugeot-207-1-4-vti/oid/3609624
https://www.index.hr/oglasi/opel-astra-classic-1-4-xep/oid/3609599
https://www.index.hr/oglasi/skoda-octavia-combi-1-6-tdi-style/oid/3609595
https://www.index.hr/oglasi/vw-up-gti/oid/3609586
https://www.index.hr/oglasi/jaguar-xe-2-0-d-r-sport-hud-panorama-line-assist-kamera-sign-recognition/oid/3609564
https://www.index.hr/oglasi/renault-clio-1-5-dci/oid/3609541
https://www.index.hr/oglasi/peugeot-308

https://www.index.hr/oglasi/chrysler-voyager-2-5-crd/oid/3605921
https://www.index.hr/oglasi/bmw-serija-4-gran-coupe-418d-m-sport/oid/3605884
https://www.index.hr/oglasi/opel-zafira-1-9-cdti-120ks/oid/3605822
https://www.index.hr/oglasi/opel-corsa-1-3-cdti/oid/3605821
https://www.index.hr/oglasi/renault-megane-scenic-1-6-dci/oid/3605813
https://www.index.hr/oglasi/opel-corsa-1-2-16v/oid/3605805
https://www.index.hr/oglasi/hyundai-ix35-1-7crdi/oid/3605761
https://www.index.hr/oglasi/ford-c-max-1-6-tdci/oid/3605747
https://www.index.hr/oglasi/vw-polo-1-4-tdi-bmt-2016g/oid/3605733
https://www.index.hr/oglasi/fiat-grande-punto-1-3-mjet/oid/3605720
https://www.index.hr/oglasi/ford-fiesta-s-1-6tdci/oid/3605682
https://www.index.hr/oglasi/yugo-tempo-1-1/oid/3605580
https://www.index.hr/oglasi/peugeot-207-1-6-hdi/oid/3605567
https://www.index.hr/oglasi/opel-corsa-1-2-16v/oid/3605552
https://www.index.hr/oglasi/bmw-serija-2-active-tourer-220-190ks-sport-line/oid/3605528
https://www.index.hr/ogl

https://www.index.hr/oglasi/vw-golf-vii-1-6-tdi-comfort-77kw/oid/3601118
https://www.index.hr/oglasi/bmw-serija-1-120d-130kw-m-paket/oid/3601062
https://www.index.hr/oglasi/opel-corsa-1-4-color-edition/oid/3601021
https://www.index.hr/oglasi/volvo-v40-d2/oid/3601020
https://www.index.hr/oglasi/volvo-xc90-2005/oid/3600986
https://www.index.hr/oglasi/suzuki-sx4-s-cross-1-0/oid/3600979
https://www.index.hr/oglasi/opel-astra-karavan-1-7-cdti/oid/3600964
https://www.index.hr/oglasi/mitsubishi-colt-1-1-benzin/oid/3600917
https://www.index.hr/oglasi/mercedes-benz-e-klasa-t-model-220cdi/oid/3600914
https://www.index.hr/oglasi/audi-a4-2-0-tdi/oid/3600849
https://www.index.hr/oglasi/opel-astra-1-6/oid/3600810
https://www.index.hr/oglasi/skoda-octavia-ii-1-6-tdi-ambiente-77kw/oid/3600538
https://www.index.hr/oglasi/audi-a3-1-9-tdi/oid/3600537
https://www.index.hr/oglasi/citroen-c2-x-1-1-2004-g/oid/3600526
https://www.index.hr/oglasi/bmw-serija-1-118d/oid/3600513
https://www.index.hr/oglasi/audi-a

In [7]:
scrape(353, 403)
df4 = pd.DataFrame(dataIndex)
df4.to_csv("rawIndex353-403savepoint.csv", index = False)

xxxxxxxxxxxxxxxxxxxxx page  353 of 
https://www.index.hr/oglasi/bmw-serija-1-118d/oid/3600100
https://www.index.hr/oglasi/renault-scenic-1-6-benzin/oid/3600094
https://www.index.hr/oglasi/bmw-serija-3-1-6-i/oid/3600078
https://www.index.hr/oglasi/hyundai-ix20-1-6-crdi-85-kw/oid/3600069
https://www.index.hr/oglasi/peugeot-3008-1-5-bluehdi/oid/3600067
https://www.index.hr/oglasi/bmw-serija-318i/oid/3600063
https://www.index.hr/oglasi/vw-passat-variant-1-6-tdi/oid/3600044
https://www.index.hr/oglasi/ford-fiesta-1-5-tdci-garancija-12-mj-moze-na-rate/oid/3600036
https://www.index.hr/oglasi/chevrolet-spark/oid/3600034
https://www.index.hr/oglasi/seat-cordoba-1-4-bezin-lpg/oid/3599913
https://www.index.hr/oglasi/nissan-qashqai-1-5-dci-acenta/oid/3599890
https://www.index.hr/oglasi/vw-golf-iv-variant-1-6-tdi/oid/3599781
https://www.index.hr/oglasi/peugeot-partner-16hdi/oid/3599763
https://www.index.hr/oglasi/honda-accord-3-5-ex-l-v6-sedan-automatik-14-000-nije-fiksno/oid/3599705
https://www.in

https://www.index.hr/oglasi/citroen-c3-1-4-hdi-medvescak/oid/3595317
https://www.index.hr/oglasi/renault-scenic-1-5-dci/oid/3595309
https://www.index.hr/oglasi/vw-beetle-1-4-tdi/oid/3595307
https://www.index.hr/oglasi/renault-clio-1-2-16v/oid/3595298
https://www.index.hr/oglasi/seat-leon-1-6-tdi/oid/3595291
https://www.index.hr/oglasi/bmw-serija-3-coupe-2-0d/oid/3595286
https://www.index.hr/oglasi/opel-astra-1-7-dti/oid/3595264
https://www.index.hr/oglasi/prodajem-automobil-renault-scenic-1-5-dci-registiran/oid/3595240
https://www.index.hr/oglasi/mercedes-benz-e-klasa-coupe-350d-full-oprema-top-aut0/oid/3595227
https://www.index.hr/oglasi/kia-ceed-1-4crdi/oid/3595219
https://www.index.hr/oglasi/audi-a3-audi-a3-2-0-tdi-125-kw/oid/3595216
https://www.index.hr/oglasi/renault-scenic-1-5-dci/oid/3595139
https://www.index.hr/oglasi/vw-bora-variant-1-9-tdi-85-kw-reg-7-23/oid/3595104
https://www.index.hr/oglasi/audi-a4b8-2-0tdi/oid/3595047
https://www.index.hr/oglasi/renault-megane-grandtour-1

https://www.index.hr/oglasi/audi-a4-avant-2-0-tdi/oid/3590342
https://www.index.hr/oglasi/skoda-octavia-1-9-tdi-bkc/oid/3590329
https://www.index.hr/oglasi/vw-golf-v-1-9-tdi/oid/3590285
https://www.index.hr/oglasi/mercedes-benz-cla-klasa-200-cdi/oid/3590264
https://www.index.hr/oglasi/vw-polo-classic-1-4/oid/3588760
https://www.index.hr/oglasi/renault-megane-1-6-diesel-96kw-gt-line/oid/3588750
https://www.index.hr/oglasi/opel-astra-sport-gtc/oid/3588747
https://www.index.hr/oglasi/opel-astra-2-0-dti/oid/3588725
https://www.index.hr/oglasi/opel-astra-h-1-7cdti-74kw-2010g-110-000km-nije-uvoz-ispisstp-servisna/oid/3588719
https://www.index.hr/oglasi/audi-a6-2-0-tdi/oid/3588715
xxxxxxxxxxxxxxxxxxxxx page  358 of 
https://www.index.hr/oglasi/audi-a6-2-0-tdi/oid/3588715
https://www.index.hr/oglasi/volvo-s40-1-9td-klima/oid/3588679
https://www.index.hr/oglasi/bmw-serija-3-e90lci/oid/3588678
https://www.index.hr/oglasi/vw-golf-vii-1-6-tdi-bmt-cup-navi-auto-pdc-park-assist/oid/3588677
https://w

https://www.index.hr/oglasi/chevrolet-captiva-2-0-d-150-tkm-hr-auto-2009-zamjena-obrocno-placanje/oid/3584736
https://www.index.hr/oglasi/skoda-fabia-1-4htp-prilika/oid/3584733
https://www.index.hr/oglasi/vw-golf-vii-1-6-tdi/oid/3584728
https://www.index.hr/oglasi/chevrolet-captiva-2-0-d-reg-5-2023-2010god/oid/3584711
https://www.index.hr/oglasi/renault-clio-1-2-16v-expression/oid/3584708
https://www.index.hr/oglasi/opel-zafira-1-8-16v-plin/oid/3584525
https://www.index.hr/oglasi/suzuki-sx4-1-5/oid/3584518
https://www.index.hr/oglasi/peugeot-308-sw-1-6-hdi/oid/3584512
https://www.index.hr/oglasi/bmw-serija-1-bmw-118-d/oid/3584482
https://www.index.hr/oglasi/peugeot-301-1-6-blue-hdi/oid/3584479
https://www.index.hr/oglasi/renault-megane-grand-coupe/oid/3583620
https://www.index.hr/oglasi/bmw-serija-1-180/oid/3583578
https://www.index.hr/oglasi/bmw-serija-5-530d/oid/3583569
https://www.index.hr/oglasi/vw-touran-1-6-benzin-plin/oid/3583555
https://www.index.hr/oglasi/mitsubishi-space-star

https://www.index.hr/oglasi/opel-kadett-b/oid/3579181
https://www.index.hr/oglasi/citroen-c5-break-1-6-tdci-automatik-navigacija-tourer-oprema/oid/3579174
https://www.index.hr/oglasi/bmw-730d-xdrive-virtual-cockpit-oba-seta-guma-facelift/oid/3579126
https://www.index.hr/oglasi/opel-kadett-cabrio-1-6i/oid/3579087
https://www.index.hr/oglasi/mercedes-benz-180-180-cdi-b-klasa/oid/3579062
https://www.index.hr/oglasi/opel-astra-coupe-z20let-turbo-bertone/oid/3579016
https://www.index.hr/oglasi/bmw-serija-3-touring-316d/oid/3578974
https://www.index.hr/oglasi/vw-transporter-t5/oid/3578893
https://www.index.hr/oglasi/opel-astra-1-7-cdti-sport/oid/3578850
https://www.index.hr/oglasi/vw-passat-1-6-tdi-trendline/oid/3578831
https://www.index.hr/oglasi/alfa-romeo-156-1-9/oid/3578408
https://www.index.hr/oglasi/bmw-serija-3-320d/oid/3578379
https://www.index.hr/oglasi/hyundai-accent-1-4/oid/3578332
https://www.index.hr/oglasi/kia-xceed-1-0-t-gdi-lx-m-t/oid/3578286
https://www.index.hr/oglasi/peuge

...

https://www.index.hr/oglasi/opel-astra-lpg/oid/3505256
https://www.index.hr/oglasi/ford-focus-1-6-benzinac/oid/3505234
https://www.index.hr/oglasi/citroen-c5-break-2-0-hdi/oid/3505216
https://www.index.hr/oglasi/bmw-serija-3-318-d/oid/3505206
https://www.index.hr/oglasi/renault-espace-2-2-dci-moze-zamjena-kombinacije-rate-full-oprema-hr-papiri-u-dijelovima/oid/3505205
https://www.index.hr/oglasi/ford-mondeo-karavan-diesel/oid/3505161
https://www.index.hr/oglasi/skoda-octavia-combi-1-9-tdi/oid/3505145
https://www.index.hr/oglasi/hyundai-galloper-2-5-dizel/oid/3505126
https://www.index.hr/oglasi/renault-vel-satis-2-2-dci-moze-zamjena-kombinacije-rate-full-oprema-nova-registracija/oid/3504662
https://www.index.hr/oglasi/ford-fiesta-1-4-tdci-moze-zamjena-kombinacije-rate-full-oprema-nova-registracija/oid/3504660
https://www.index.hr/oglasi/bmw-740-d-moze-zamjena-kombinacije-rate-full-max-oprema-nova-registracija/oid/3504658
https://www.index.hr/oglasi/vw-golf-vii-1-6tdi-lounge/oid/3504634


https://www.index.hr/oglasi/opel-astra-sve-marke-i-tipove-vozila-nebitno-u-kakvom-su-stanju-otkup-i-odvoz-auta-za-otpad-reciklazu-tel-0919747009/oid/3500643
https://www.index.hr/oglasi/citroen-xsara-1-4-i/oid/3500626
https://www.index.hr/oglasi/bmw-serija-3-320-d/oid/3500595
https://www.index.hr/oglasi/lancia-delta-1-6-multijet/oid/3500563
https://www.index.hr/oglasi/vw-golf-vii-1-6-tdi-dsg/oid/3500535
https://www.index.hr/oglasi/mercedes-benz-e-klasa-250/oid/3500507
https://www.index.hr/oglasi/vw-polo-1-4-tdi-bmt-2016/oid/3500477
https://www.index.hr/oglasi/bmw-serija-5-bmw-e39-525d-120kw/oid/3500449
https://www.index.hr/oglasi/bmw-serija-3-318d-m-sport-paket/oid/3500448
https://www.index.hr/oglasi/ford-fiesta/oid/3500400
https://www.index.hr/oglasi/audi-a3-sportback-2-0-tdi-automatic/oid/3500389
https://www.index.hr/oglasi/peugeot-206-2-0-hdi-reg-godinu-dana/oid/3500324
https://www.index.hr/oglasi/citroen-c3-1-6-hdi/oid/3500302
https://www.index.hr/oglasi/mercedes-benz-ml/oid/3500215

https://www.index.hr/oglasi/mercedes-benz-e-klasa-320-cdi/oid/3494976
https://www.index.hr/oglasi/seat-ibiza-1-0-tsi/oid/3494872
https://www.index.hr/oglasi/peugeot-206-1-4e-benzin-2001-godiste-crni/oid/3494855
https://www.index.hr/oglasi/renault-grand-scenic-1-5-dci-2013-godina-7sjedala-1-5-dci-110-ks/oid/3494851
https://www.index.hr/oglasi/citroen-xsara-2-0-hdi/oid/3494805
https://www.index.hr/oglasi/vw-golf-vii-1-6-tdi/oid/3494789
https://www.index.hr/oglasi/vw-passat-1-9-tdi/oid/3494735
xxxxxxxxxxxxxxxxxxxxx page  398 of 
https://www.index.hr/oglasi/vw-passat-1-9-tdi/oid/3494735
https://www.index.hr/oglasi/toyota-yaris-1-0-vvt-i/oid/3494687
https://www.index.hr/oglasi/honda-civic/oid/3494661
https://www.index.hr/oglasi/bmw-serija-3-touring-320-d/oid/3494653
https://www.index.hr/oglasi/audi-a4-2-0-tdi/oid/3494615
https://www.index.hr/oglasi/audi-a3-1-6-tdi-automatski-mjenjac-u-odlicnom-stanju/oid/3494609
https://www.index.hr/oglasi/skoda-octavia-1-6-automatik/oid/3494593
https://www

https://www.index.hr/oglasi/ford-fiesta-1-6-tdci-titanium/oid/3490189
https://www.index.hr/oglasi/opel-astra-1-8-16v-gsi/oid/3490165
https://www.index.hr/oglasi/nema-krize-atestiran-plin-super-prilika-mazda-cx-7-2-3-turbo-4x4-2008-g-reg-9-22-perla-bijela-koza-alu-18-keyless-xenon-pdc-samo-4999-gotovine/oid/3490107
https://www.index.hr/oglasi/vw-passat-cc-1-8-tsi-2008g-reg-11-22-alu-18-koza-pdc-eps-mf-volan-cornering-light-prilika-samo-6500-gotovine/oid/3490080
https://www.index.hr/oglasi/mercedes-benz-180-cgi/oid/3490022
https://www.index.hr/oglasi/renault-megane-1-5-dci/oid/3489178
https://www.index.hr/oglasi/hyundai-i30-1-4-dohc-16-v-registriran-do-01-23/oid/3489177
https://www.index.hr/oglasi/renault-megane-coupe-1-6e/oid/3489137
https://www.index.hr/oglasi/bmw-serija-1/oid/3177258
https://www.index.hr/oglasi/mercedes-benz-c-klasa-1995/oid/3489072
https://www.index.hr/oglasi/opel-insignia-2-0/oid/3489042
https://www.index.hr/oglasi/audi-a1-1-6-tdi/oid/3489039
https://www.index.hr/og

https://www.index.hr/oglasi/toyota-yaris-1-4-d-4d-dizel/oid/3482484
https://www.index.hr/oglasi/vw-golf-vii-2-0-tdi/oid/3482300
https://www.index.hr/oglasi/bmw-e36-cabriolet-m-paket/oid/3482295
https://www.index.hr/oglasi/audi-a6-avant-3-0tdi/oid/3482242
https://www.index.hr/oglasi/opel-vectra-2-2-automatik/oid/3481980
https://www.index.hr/oglasi/citroen-c4-1-6-hdi-runner-moze-zamjena/oid/3481970
https://www.index.hr/oglasi/peugeot-407-1-8/oid/3481941
https://www.index.hr/oglasi/opel-zafira-1-6-cdti/oid/3481913
https://www.index.hr/oglasi/vw-passat-cc-2-0-tsi/oid/3481908
https://www.index.hr/oglasi/audi-a8-3-0tdi/oid/3481857
https://www.index.hr/oglasi/bmw-serija-5-2-5-benzin/oid/3481842
https://www.index.hr/oglasi/kia-cerato-1-5-crdi/oid/3481835
https://www.index.hr/oglasi/bmw-serija-3-bmw-f30-318d/oid/3481739
https://www.index.hr/oglasi/mercedes-benz-e-klasa-220-dizel/oid/3481684
https://www.index.hr/oglasi/renault-clio-1-2/oid/3481683
https://www.index.hr/oglasi/skoda-superb-2-0-tdi

In [7]:
scrape(403, 442)
df5 = pd.DataFrame(dataIndex)
df5.to_csv("rawIndex403-442.csv", index = False)

xxxxxxxxxxxxxxxxxxxxx page  403 of 
https://www.index.hr/oglasi/fiat-bravo-1-4-multiair-turbo/oid/3481599
https://www.index.hr/oglasi/audi-a6-avant-2-5-tdi-diesel-114-kw/oid/3481563
https://www.index.hr/oglasi/opel-astra-karavan-k-1-6-cdti/oid/3481552
https://www.index.hr/oglasi/pontiac-firebird-3-4-benzin-lpg/oid/3481536
https://www.index.hr/oglasi/opel-corsa-1-2-16v-lpg/oid/3481501
https://www.index.hr/oglasi/ford-c-max/oid/3481498
https://www.index.hr/oglasi/skoda-octavia-1-6tdi/oid/3481491
https://www.index.hr/oglasi/nissan-qashqai-1-6dci-130ks-acenta/oid/3481489
https://www.index.hr/oglasi/skoda-octavia-combi-1-9-tdi/oid/3481473
https://www.index.hr/oglasi/peugeot-206-1-4/oid/3481465
https://www.index.hr/oglasi/vw-polo-1-6tdi-2011g-reg-5-23/oid/3481434
https://www.index.hr/oglasi/seat-leon-1-6-tdi-2017godina/oid/3481426
https://www.index.hr/oglasi/bmw-serija-3-318d-xdrive/oid/3481422
https://www.index.hr/oglasi/mercedes-benz-a-klasa-a180-cdi/oid/3481412
https://www.index.hr/oglasi

https://www.index.hr/oglasi/renault-clio-1-5-dci-2005-g/oid/3474228
https://www.index.hr/oglasi/audi-a5-audi-a5-sportback-2-0-tdi-automatik/oid/3474219
https://www.index.hr/oglasi/mazda-6-2-0-89kw/oid/3474194
https://www.index.hr/oglasi/renault-clio-1-5-dci/oid/3473965
https://www.index.hr/oglasi/vw-tiguan-2-0-tdi-bmt/oid/3473939
https://www.index.hr/oglasi/mazda-b-2500-4x4/oid/3473918
https://www.index.hr/oglasi/peugeot-307-20hdi/oid/3473871
https://www.index.hr/oglasi/vw-polo-1-2-tdi-match/oid/3473108
https://www.index.hr/oglasi/renault-scenic-1-5-dci/oid/3473057
https://www.index.hr/oglasi/opel-astra-karavan-1-6-cdti-2015-100-kw-cosmo-euro-6/oid/3473047
https://www.index.hr/oglasi/renault-laguna-2-0-ide/oid/3473037
https://www.index.hr/oglasi/opel-corsa-1-3-cdti-3-vrata-cosmo/oid/3472965
https://www.index.hr/oglasi/vw-golf-iv-1-9-tdi/oid/3472942
https://www.index.hr/oglasi/mercedes-benz-c-klasa-c220-cdi-avantgarde-automatic/oid/3472931
https://www.index.hr/oglasi/renault-megane-seda

https://www.index.hr/oglasi/mercedes-benz-c-klasa-t-model-220-cdi-t-automatik/oid/3467223
https://www.index.hr/oglasi/kia-pro-ceed-1-6-benz/oid/3467217
https://www.index.hr/oglasi/mercedes-benz-b-klasa-180cdi/oid/3467215
https://www.index.hr/oglasi/renault-grand-scenic-1-9-cdti/oid/3467212
https://www.index.hr/oglasi/mazda-626-2-0-lpg/oid/3467174
https://www.index.hr/oglasi/audi-a3-1-6/oid/3467107
https://www.index.hr/oglasi/vw-passat-2-0-njemacki-uvoz/oid/3467068
xxxxxxxxxxxxxxxxxxxxx page  408 of 
https://www.index.hr/oglasi/renault-megane-break-1-5dci/oid/3467065
https://www.index.hr/oglasi/bmw-serija-1-116d-sport/oid/3466951
https://www.index.hr/oglasi/peugeot-308-1-6-vvti/oid/3466920
https://www.index.hr/oglasi/ford-focus-1-6-ti-sport/oid/3466849
https://www.index.hr/oglasi/opel-insignia-eco-flex-cdti-2-0-2016/oid/3466830
https://www.index.hr/oglasi/vw-up-move-up/oid/3466727
https://www.index.hr/oglasi/volvo-xc60-xc-60-d4-inscription-apsolutno-full-u-sustavu-pdv-a-u-dolasku/oid/34

https://www.index.hr/oglasi/fiat-punto-evo-1-3-multijet/oid/3458517
https://www.index.hr/oglasi/audi-a6-avant-2-5tdi/oid/3458501
https://www.index.hr/oglasi/kia-cerato-1-5-dizel/oid/3458453
https://www.index.hr/oglasi/2020-mini-cooper-se-electric/oid/3457327
https://www.index.hr/oglasi/fiat-punto-1-2-sx/oid/3457326
https://www.index.hr/oglasi/vw-passat-variant-2-0/oid/3457266
https://www.index.hr/oglasi/bmw-serija-3-bmw-e36-1-8/oid/3457171
https://www.index.hr/oglasi/kia-pro-ceed-1-4/oid/3457066
https://www.index.hr/oglasi/audi-a4-avant-35-tdi-ultra/oid/3457052
https://www.index.hr/oglasi/ford-kuga-1-5-tdci-powershift-automatik-econetic-business-facelift/oid/3457014
https://www.index.hr/oglasi/skoda-octavia-1-9-tdi/oid/3456927
https://www.index.hr/oglasi/audi-a4-avant-2-0-tdi-s-tronik/oid/3456877
https://www.index.hr/oglasi/opel-vectra/oid/3456863
https://www.index.hr/oglasi/vw-fox/oid/3456746
https://www.index.hr/oglasi/peugeot-508-1-6-e-hdi/oid/3456744
https://www.index.hr/oglasi/ren

https://www.index.hr/oglasi/vw-golf-vii-gtd-prilika/oid/3450815
https://www.index.hr/oglasi/mercedes-benz-e-klasa-220-cdi/oid/3450814
https://www.index.hr/oglasi/vw-golf-v-1-9-tdi/oid/3450808
https://www.index.hr/oglasi/vw-polo-1-4-gti-16v/oid/3450779
https://www.index.hr/oglasi/audi-a3-sportback-2-0-tdi/oid/3450768
https://www.index.hr/oglasi/chevrolet-captiva-2-2-tdi/oid/3450684
https://www.index.hr/oglasi/opel-astra-1-7-cdti/oid/3450673
https://www.index.hr/oglasi/bmw-serija-5-530d-e39/oid/3450669
https://www.index.hr/oglasi/peugeot-207-1-4-hdi/oid/3450656
https://www.index.hr/oglasi/vw-polo-1-6tdi/oid/3450576
https://www.index.hr/oglasi/renault-clio-ripcul-1-5dci-2007-god/oid/3450572
https://www.index.hr/oglasi/citroen-c4-1-6-16-v/oid/3450543
https://www.index.hr/oglasi/opel-corsa-1-2-1-vlasnica/oid/3450490
https://www.index.hr/oglasi/mercedes-benz-a-klasa-250e-hibrid/oid/3450467
https://www.index.hr/oglasi/seat-ibiza-1-4-tdi-fr-look/oid/3450446
https://www.index.hr/oglasi/vw-golf-

...

https://www.index.hr/oglasi/ford-grand-c-max-1-6-tdci/oid/3118070
https://www.index.hr/oglasi/fiat-stilo-1-2-16v/oid/3116359
https://www.index.hr/oglasi/mazda-3-1-6-d/oid/3115324
https://www.index.hr/oglasi/mazda-3-1-6-d/oid/3115319
https://www.index.hr/oglasi/fiat-barchetta/oid/3115058
https://www.index.hr/oglasi/auto-otkup-zagreb/oid/3115002
https://www.index.hr/oglasi/vw-golf-vii-1-6-automatic/oid/3114247
https://www.index.hr/oglasi/mazda-323-sedan-1-3-i/oid/3113597
https://www.index.hr/oglasi/mercedes-benz-e-klasa-220-cdi-amg-full-u-pdv-u/oid/3111649
https://www.index.hr/oglasi/bmw-serija-3-318i-105kw-143ks/oid/3109683
https://www.index.hr/oglasi/bmw-x5-f15-3-0d/oid/3107542
https://www.index.hr/oglasi/renault-clio-renault-clio-1-5-dci-2011-god/oid/3103895
https://www.index.hr/oglasi/mercedes-benz-viano-2-2-cdi-110-kw/oid/3103432
https://www.index.hr/oglasi/peugeot-207-cc-16-16v-plin-reg-15-09-22/oid/3102310
https://www.index.hr/oglasi/smart-fortwo-coupe-prime-prvi-vlasnik-2016-godi

https://www.index.hr/oglasi/opel-zafira-1-9-cdti/oid/2953445
https://www.index.hr/oglasi/skoda-octavia-combi-tdi-novi-zupcasti-puno-opreme/oid/2951950
https://www.index.hr/oglasi/jaguar-s-type-r-4-2v8-s-c/oid/2951493
https://www.index.hr/oglasi/bmw-serija-3-318i/oid/2837093
https://www.index.hr/oglasi/audi-a2-1-4-tdi-66-kw-2004-godina/oid/2941925
https://www.index.hr/oglasi/volvo-v50-2-0-tdi-reg-1-g/oid/2730424
https://www.index.hr/oglasi/chevrolet-orlando-2-0-td-lt/oid/2938407
https://www.index.hr/oglasi/mercedes-benz-e-klasa-t-model-250-d-1998-god-cjena-je-na-ime-kupca/oid/2933667
https://www.index.hr/oglasi/bmw-serija-3-318d/oid/2933043
https://www.index.hr/oglasi/peugeot-308-sw-1-6-blue-hdi/oid/2931636
https://www.index.hr/oglasi/bmw-x3-2-0/oid/2928434
https://www.index.hr/oglasi/peugeot-508-sw-2-2-hdi-gt/oid/2928249
https://www.index.hr/oglasi/mini-one-14-diesel/oid/2927117
https://www.index.hr/oglasi/vw-passat-variant-2-0-tdi-bmt/oid/2926707
https://www.index.hr/oglasi/smart-fort

https://www.index.hr/oglasi/mercedes-benz-e-klasa-220cdi/oid/2755334
https://www.index.hr/oglasi/vw-passat-variant-1-6-tdi/oid/2749480
https://www.index.hr/oglasi/renault-megane-scenic-1-5-dci-2014-g-100-tkm-2-vlasnik-zg-garaziran-zamjena-za-novije/oid/2743785
https://www.index.hr/oglasi/opel-astra-classic-1-4-twinport-benzin-plin-reg-do-06-2023/oid/2742538
https://www.index.hr/oglasi/fiat-500-fiat-500-1-2-8v-sa-2-seta-guma/oid/2740900
https://www.index.hr/oglasi/vw-passat-2-0-tdi-registriran-do-12-6-2023/oid/2740845
https://www.index.hr/oglasi/bmw-serija-3-touring-325d-lci/oid/2740626
https://www.index.hr/oglasi/toyota-avensis-wagon-touring-sports-2-0-d-4d/oid/2740358
https://www.index.hr/oglasi/peugeot-407-1-6-hdi-reg-2-24/oid/2739834
https://www.index.hr/oglasi/vw-caravelle-2-0-tdi/oid/2736315
https://www.index.hr/oglasi/peugeot-206-1-4-hdi-reg-1-god/oid/2727663
xxxxxxxxxxxxxxxxxxxxx page  436 of 
https://www.index.hr/oglasi/bmw-serija-4-gran-coupe-420-d-automatic/oid/2723168
https:

https://www.index.hr/oglasi/audi-a6-2-0-s-tronic-black-edition/oid/2385418
https://www.index.hr/oglasi/vw-golf-ii-ic-1-6-gtd/oid/2385318
https://www.index.hr/oglasi/audi-a4-avant-2-0-tdi/oid/1265948
https://www.index.hr/oglasi/mercedes-benz-c-klasa-c200-cdi-w203/oid/2381026
https://www.index.hr/oglasi/mercedes-benz-a-klasa-170/oid/2371202
https://www.index.hr/oglasi/ford-escort-cabriolet-rs/oid/2367714
https://www.index.hr/oglasi/renault-vel-satis-3-5-v6-plin/oid/2364536
https://www.index.hr/oglasi/citroen-c5-1-6-hdi/oid/2351673
https://www.index.hr/oglasi/citroen-xsara-picasso-1-6hdi/oid/2350121
https://www.index.hr/oglasi/vw-golf-iv-19sdi/oid/2350120
https://www.index.hr/oglasi/mercedes-benz-b-klasa-180cdi/oid/2350104
https://www.index.hr/oglasi/toyota-avensis-2-0d-kat/oid/2350092
https://www.index.hr/oglasi/mini-one/oid/2347507
https://www.index.hr/oglasi/alfa-romeo-159-1-9-jtdm-16v/oid/2347449
https://www.index.hr/oglasi/bmw-serija-3-320d-luxury/oid/2339943
https://www.index.hr/ogl

https://www.index.hr/oglasi/fiat-panda-1-2-klima-55000-tk-reg-godinu-dana/oid/1829904
https://www.index.hr/oglasi/peugeot-3008-1-6-e-hdi-business-pack/oid/1819502
https://www.index.hr/oglasi/renault-megane-grandtour-1-5-dci-bose-edition/oid/1812982
https://www.index.hr/oglasi/vw-golf-vii-1-9-tdi-comfortline/oid/1809850
https://www.index.hr/oglasi/ford-fiesta-1-4-tdci-2008g-reg-03-2024/oid/1775095
https://www.index.hr/oglasi/saab-9-5-1-9/oid/1774645
https://www.index.hr/oglasi/honda-jazz-1-4-dsi-originalno-stanje-1-vlasnik-reg-1-godinu/oid/1744707
https://www.index.hr/oglasi/dodge-nitro-2-8-crd-4x4-automatik/oid/1736933
https://www.index.hr/oglasi/bmw-x3-2-0d-xline/oid/1726774
https://www.index.hr/oglasi/audi-a4-2-0-tdi/oid/1706434
https://www.index.hr/oglasi/vw-golf-vi-1-6-tdi-bluemotion/oid/1676604
https://www.index.hr/oglasi/mercedes-benz-e320-cdi-karavan/oid/1646875
https://www.index.hr/oglasi/opel-astra-karavan-1-7-dti/oid/1619257
https://www.index.hr/oglasi/mercedes-benz-e-220/oid

In [5]:
print(lastPageScraped)

None


In [6]:
print(dataIndex)

[]


## Merging index.com files

In [1]:
import pandas as pd

In [3]:
part1 = pd.read_csv("savepoints/rawIndex1-100.csv")

In [5]:
part2 = pd.read_csv("savepoints/rawIndex102-202.csv")

In [6]:
part3 = pd.read_csv("savepoints/rawIndex203-303savepoint.csv")

In [7]:
part4 = pd.read_csv("savepoints/rawIndex303-353savepoint.csv")

In [8]:
part5 = pd.read_csv("savepoints/rawIndex353-403savepoint.csv")

In [9]:
part6 = pd.read_csv("savepoints/rawIndex403-442.csv")

In [10]:
merged = pd.concat([part1, part2, part3, part4, part5, part6], ignore_index = True)

In [11]:
display(merged)

,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Starost,Vlasnik,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3
0,Osijek,29.03.2023,18,13.600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215.000,2018,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN
1,Labin,29.03.2023,9,7.500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304.730,2011,...,Rabljeno,Drugi,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN
2,Sveta Nedelja,29.03.2023,8,7.700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157.000,2012,...,Rabljeno,Prvi,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN
3,Črnomerec,29.03.2023,17,2.750,Peugeot 207 1.4\r\n,1.4,Benzin + LPG,Uredno,190.000,2007,...,Rabljeno,Drugi,Prednji,gotovina,Prodajem jer je višak,Mehanički,NaN,NaN,NaN,NaN
4,Slatina,29.03.2023,20,5.700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385.000,2009,...,Rabljeno,Drugi,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21901,Osijek,25.06.2017,7517,5.7,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525.000,2007,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN
21902,Sisak,08.02.2017,10136,4.5,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255.000,2007,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN
21903,Nova cesta,03.02.2017,852,8.5,Volvo S80 2.4D\r\n,2.4D,Diesel,Uredno,240.000,2008,...,Rabljeno,Drugi,Prednji,gotovina,NaN,Automatski,NaN,Volvo,S80,NaN
21904,Podsused - Vrapče,01.01.2017,23049,11.3,VW Golf V GTI\r\n,GTI,Benzin,Kao novo,146.200,2008,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,VW,Golf V,NaN


In [12]:
merged.to_csv("rawCsvData/rawIndex.csv", index = False)

<h2>Data Cleaning</h2>

I have a decent amount of cars now from two different ad sites. Now I need to load them and get the most out of the data that I've scraped.

In [1]:
import numpy as np
import pandas as pd
import re

from sqlalchemy import create_engine
from tqdm.auto import tqdm
from unidecode import unidecode

In [2]:
pd.options.display.float_format = '{:.2f}'.format

### Location

The first thing I'm going to tackle is Location for both datasets because I know from the scraping that it's quite messy. 
The idea is to get a town/city column and a county column for each car and to drop those cars with the location unknown and undiscoverable or abroad. 
I'll of course look at the amount of cars I'd have to drop and decide if it's too much and maybe the location itself should be dropped. However, this is the worst case scenario because I think that the analysis based on location is valuable and I'll do my best to have the info on the location of each car.

This is the excel file that's going to help me extract valid names for Croatian towns and counties from location records as well as map towns to their counties in the cases where I have the info on the town where the car is but not on the county. This file will help me solve that easy problem and save some cars for getting dropped for such a trivial thing.

In [3]:
locations_df = pd.read_excel("helperData/counties-cities.xlsx")

In [4]:
locations_df

,County,Zupanija,City/Town,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Zagreb County,Zagrebačka,Dugo Selo,NaN,NaN,NaN
1,Zagreb County,Zagrebačka,Ivanić Grad,NaN,NaN,NaN
2,Zagreb County,Zagrebačka,Jastrebarsko,NaN,NaN,NaN
3,Zagreb County,Zagrebačka,Samobor,NaN,NaN,NaN
4,Zagreb County,Zagrebačka,Sveta Nedelja,NaN,NaN,NaN
...,...,...,...,...,...,...
550,Međimurje County,Međimurska,Sveti Juraj na Bregu,NaN,NaN,NaN
551,Međimurje County,Međimurska,Sveti Martin na Muri,NaN,NaN,NaN
552,Međimurje County,Međimurska,Šenkovec,NaN,NaN,NaN
553,Međimurje County,Međimurska,Štrigova,NaN,NaN,NaN


I have random 3 columns that I don't need so I'll delete that.

In [5]:
locations_df = locations_df.drop(locations_df.columns[3:6], axis=1)

I'll add new columns with cities lowercase and without diacritics because I'll need that for the Index dataframe.

In [6]:
locations_df['Town_Low'] = locations_df['City/Town'].apply(lambda x: unidecode(x).lower())
locations_df

,County,Zupanija,City/Town,Town_Low
0,Zagreb County,Zagrebačka,Dugo Selo,dugo selo
1,Zagreb County,Zagrebačka,Ivanić Grad,ivanic grad
2,Zagreb County,Zagrebačka,Jastrebarsko,jastrebarsko
3,Zagreb County,Zagrebačka,Samobor,samobor
4,Zagreb County,Zagrebačka,Sveta Nedelja,sveta nedelja
...,...,...,...,...
550,Međimurje County,Međimurska,Sveti Juraj na Bregu,sveti juraj na bregu
551,Međimurje County,Međimurska,Sveti Martin na Muri,sveti martin na muri
552,Međimurje County,Međimurska,Šenkovec,senkovec
553,Međimurje County,Međimurska,Štrigova,strigova


In [7]:
towns = locations_df['City/Town'].unique()
towns_low = locations_df['Town_Low'].unique()
counties = locations_df['Zupanija'].unique()

#### Oglasnik Data

In [8]:
df_o = pd.read_csv("rawCsvData/rawOglasnik.csv", encoding="utf-8")

In [9]:
df_o

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Pogon,Klima_uređaj,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer
0,"Sesvete, Grad Zagreb",29.03.2023.,4,1.000,VW,Golf IV,1998.00,370000 km,07/2023,74 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Novi Zagreb - Istok, Grad Zagreb",29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015.00,128400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Novi Zagreb, Grad Zagreb",29.03.2023.,264,31.000,Škoda,Kodiaq,2019.00,63400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Novi Zagreb, Grad Zagreb",29.03.2023.,330,24.400,VW,Tiguan,2017.00,113600 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Grad Zagreb,29.03.2023.,196,5.500,Dacia,Logan,2015.00,257000 km,04/2023,66 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,"Arsenal, Pula, Istarska",08.01.2021.,540,"11.533,61",NaN,NaN,2018.00,164118 km,NaN,81 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4093,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,402,17.400,NaN,NaN,2017.00,132210 km,NaN,125 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4094,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,682,21.900,NaN,NaN,2017.00,92908 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4095,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,634,39.500,NaN,NaN,2017.00,96524 km,NaN,132 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_o['Location'].unique()

array(['\xa0 Sesvete, Grad Zagreb',
       '\xa0 Novi Zagreb - Istok, Grad Zagreb',
       '\xa0 Novi Zagreb, Grad Zagreb', '\xa0 Grad Zagreb',
       '\xa0 Sisačko-moslavačka',
       '\xa0 Koprivnica, Koprivničko-križevačka',
       '\xa0 Virovitičko-podravska',
       '\xa0 Velika Gorica - Okolica, Zagrebačka',
       '\xa0 Sveti Ivan Zelina, Zagrebačka',
       '\xa0 Črnomerec, Grad Zagreb', '\xa0 Zagrebačka',
       '\xa0 Karlovačka', '\xa0 Krapinsko-zagorska',
       '\xa0 Trešnjevka - Sjever, Grad Zagreb',
       '\xa0 Podsljeme, Grad Zagreb', '\xa0 Našice, Osječko-baranjska',
       '\xa0 Krapinske Toplice, Krapinsko-zagorska',
       '\xa0 Bjelovarsko-bilogorska', '\xa0 Stenjevec, Grad Zagreb',
       '\xa0 Grubišno Polje, Bjelovarsko-bilogorska',
       '\xa0 Kneževi Vinogradi, Osječko-baranjska', '\xa0 Istarska',
       '\xa0 Gospić, Ličko-senjska', '\xa0 Trešnjevka - Jug, Grad Zagreb',
       '\xa0 Sisak, Sisačko-moslavačka', '\xa0 Vrbovec, Zagrebačka',
       '\xa0 Kutina,

After inspecting the locations, I realised that the locations are pretty clear with some of them clearly missing. (The location is: "<img alt="Telefon" src="https://www.oglasnik.hr/assets/img/icn_telephone.png"/>". I'm not sure if it's my fault for not scraping the data precisely or just that the location was not provided but I won't worry about that and just deal with it now.)



In [11]:
#Adding two new columns. The plan is to eventually drop 'Locations' column
new_df_o = df_o.copy()

new_df_o['Town'] = pd.Series([])
new_df_o['County'] = pd.Series([])

C:\Users\saind\AppData\Local\Temp\ipykernel_24456\160793083.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df_o['Town'] = pd.Series([])
C:\Users\saind\AppData\Local\Temp\ipykernel_24456\160793083.py:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df_o['County'] = pd.Series([])


In [12]:
#Regex to match towns/cities and counties inside location strings
pattern_town = r"\b{}\b".format(r"\b|\b".join(map(re.escape, towns)))
pattern_county = r"\b{}\b".format(r"\b|\b".join(map(re.escape, counties)))

#Extracts the part of the location that matches a city or a town in croatia and assigns that value to the new column 'Town'.
#For now the non-matches get assigned 'Unknown' as the town.
#Then it does the same for the county but assigns 'None' instead of 'Unknown'
for index, row in new_df_o.iterrows():
    town_match = re.search(pattern_town, row['Location'])
    town_text = town_match.group() if town_match else 'Unknown'
    new_df_o.at[index, 'Town'] = town_text
    
    county_match = re.search(pattern_county, row['Location'])
    county_text = county_match.group() if county_match else None
    new_df_o.at[index, 'County'] = county_text

new_df_o

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer,Town,County
0,"Sesvete, Grad Zagreb",29.03.2023.,4,1.000,VW,Golf IV,1998.00,370000 km,07/2023,74 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,NaN
1,"Novi Zagreb - Istok, Grad Zagreb",29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015.00,128400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,NaN
2,"Novi Zagreb, Grad Zagreb",29.03.2023.,264,31.000,Škoda,Kodiaq,2019.00,63400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,NaN
3,"Novi Zagreb, Grad Zagreb",29.03.2023.,330,24.400,VW,Tiguan,2017.00,113600 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,NaN
4,Grad Zagreb,29.03.2023.,196,5.500,Dacia,Logan,2015.00,257000 km,04/2023,66 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,"Arsenal, Pula, Istarska",08.01.2021.,540,"11.533,61",NaN,NaN,2018.00,164118 km,NaN,81 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pula,Istarska
4093,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,402,17.400,NaN,NaN,2017.00,132210 km,NaN,125 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,None
4094,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,682,21.900,NaN,NaN,2017.00,92908 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,None
4095,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,634,39.500,NaN,NaN,2017.00,96524 km,NaN,132 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,None


I'll now look at the records where County is None. If there's a town recorded, I'll match it with it's county and if there 
is no town either, then I clearly have no location for the car and I'll drop it.

In [13]:
none_county = new_df_o[new_df_o['County'].isnull()]
none_county['Town'].unique()

array(['Zagreb', 'Unknown', 'Donja Dubrava', 'Dubrava'], dtype=object)

I'll match Zagreb, Dubrava and Donja Dubrava with respective counties and drop all the rows with the town unknown.

In [14]:
print(none_county[none_county['Town'] == 'Donja Dubrava']['Location'].unique())
print(none_county[none_county['Town'] == 'Dubrava']['Location'].unique())

['\xa0 Donja Dubrava, Grad Zagreb ' '\xa0 Donja Dubrava, Grad Zagreb']
['\xa0 Gornja Dubrava, Grad Zagreb']


All occurencies of Dubrava clearly just refer to Zagreb and not to Dubrava somwehere else.
All the unmatched locations get the location of Zagreb, Zagrebačka.

In [15]:
cond = new_df_o['Town'] == 'Donja Dubrava'
new_df_o.loc[cond, 'Town'] = 'Zagreb'

In [16]:
cond2 = new_df_o['Town'] == 'Dubrava'
new_df_o.loc[cond2, 'Town'] = 'Zagreb'

In [17]:
new_df_o.loc[new_df_o['Town'] == 'Zagreb', 'County'] = 'Zagrebačka'

In [18]:
new_df_o[new_df_o['County'].isnull()]['Town'].unique() 

array(['Unknown'], dtype=object)

Now that I'm left with only unknown locations I'll drop those rows.

In [19]:
new_df_o = new_df_o.dropna(subset=['County'])

In [20]:
new_df_o

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer,Town,County
0,"Sesvete, Grad Zagreb",29.03.2023.,4,1.000,VW,Golf IV,1998.00,370000 km,07/2023,74 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
1,"Novi Zagreb - Istok, Grad Zagreb",29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015.00,128400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
2,"Novi Zagreb, Grad Zagreb",29.03.2023.,264,31.000,Škoda,Kodiaq,2019.00,63400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3,"Novi Zagreb, Grad Zagreb",29.03.2023.,330,24.400,VW,Tiguan,2017.00,113600 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
4,Grad Zagreb,29.03.2023.,196,5.500,Dacia,Logan,2015.00,257000 km,04/2023,66 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,"Arsenal, Pula, Istarska",08.01.2021.,540,"11.533,61",NaN,NaN,2018.00,164118 km,NaN,81 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pula,Istarska
4093,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,402,17.400,NaN,NaN,2017.00,132210 km,NaN,125 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
4094,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,682,21.900,NaN,NaN,2017.00,92908 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
4095,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,634,39.500,NaN,NaN,2017.00,96524 km,NaN,132 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka


This looks fine to me. Now I only need to check how many cars I lost.

In [21]:
len(df_o) - len(new_df_o)

698

It's definitely not nothing but I still have 3399 cars left in this dataset and I have the location info on all of them so I've decided to keep the new dataframe.

In [22]:
df_o = new_df_o.reset_index(drop=True).copy()

In [23]:
df_o

,Location,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,...,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer,Town,County
0,"Sesvete, Grad Zagreb",29.03.2023.,4,1.000,VW,Golf IV,1998.00,370000 km,07/2023,74 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
1,"Novi Zagreb - Istok, Grad Zagreb",29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015.00,128400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
2,"Novi Zagreb, Grad Zagreb",29.03.2023.,264,31.000,Škoda,Kodiaq,2019.00,63400 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3,"Novi Zagreb, Grad Zagreb",29.03.2023.,330,24.400,VW,Tiguan,2017.00,113600 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
4,Grad Zagreb,29.03.2023.,196,5.500,Dacia,Logan,2015.00,257000 km,04/2023,66 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,"Arsenal, Pula, Istarska",08.01.2021.,540,"11.533,61",NaN,NaN,2018.00,164118 km,NaN,81 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pula,Istarska
3395,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,402,17.400,NaN,NaN,2017.00,132210 km,NaN,125 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3396,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,682,21.900,NaN,NaN,2017.00,92908 km,NaN,110 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3397,"Jankomir, Stenjevec, Grad Zagreb",08.01.2021.,634,39.500,NaN,NaN,2017.00,96524 km,NaN,132 kW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka


#### Index Data

I'll try and do the same for this dataset as well, maybe using a slightly different approach since this dataset is much larger than the Oglasnik one.

In [24]:
df_i = pd.read_csv("data/indexCorrected.csv", encoding="utf-8")
df_i

C:\Users\saind\AppData\Local\Temp\ipykernel_24456\1979384942.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_i = pd.read_csv("data/indexCorrected.csv", encoding="utf-8")


,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Starost,Vlasnik,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3
0,Osijek,29.03.2023,18,13600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215000.00,2018,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN
1,Labin,29.03.2023,9,7500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304730.00,2011,...,Rabljeno,Drugi,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN
2,Sveta Nedelja,29.03.2023,8,7700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157000.00,2012,...,Rabljeno,Prvi,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN
3,Črnomerec,29.03.2023,17,2750,Peugeot 207 1.4\r\n,1.4,Benzin + LPG,Uredno,190000.00,2007,...,Rabljeno,Drugi,Prednji,gotovina,Prodajem jer je višak,Mehanički,NaN,NaN,NaN,NaN
4,Slatina,29.03.2023,20,5700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385000.00,2009,...,Rabljeno,Drugi,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21901,Osijek,25.06.2017,7517,5700,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525000.00,2007,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN
21902,Sisak,08.02.2017,10136,4500,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255000.00,2007,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN
21903,Nova cesta,03.02.2017,852,8500,Volvo S80 2.4D\r\n,2.4D,Diesel,Uredno,240000.00,2008,...,Rabljeno,Drugi,Prednji,gotovina,NaN,Automatski,NaN,Volvo,S80,NaN
21904,Podsused - Vrapče,01.01.2017,23049,11300,VW Golf V GTI\r\n,GTI,Benzin,Kao novo,146200.00,2008,...,Rabljeno,Drugi,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,VW,Golf V,NaN


In [25]:
df_i["Location"].unique()

array(['Osijek', 'Labin', 'Sveta Nedelja', 'Črnomerec', 'Slatina',
       'Virovitica', 'Zadar', 'Draganić', 'Novi Zagreb - Zapad',
       'Stenjevec', 'Koprivnica', 'Kaštela', 'Našice', 'Trilj', 'Sesvete',
       'Peščenica - Žitnjak', 'Donja Dubrava', 'Pirovac', 'Nova Rača',
       'Gornja Dubrava', 'Velika Gorica - Okolica', 'Velika Gorica',
       'Dugo Selo', 'Novi Zagreb - Istok', 'XIV. Podbrežje\r\n2',
       'Crikvenica', 'Poreč', 'Karlovac', 'Šumet 22\r\n20236',
       'Zagreb - Okolica', 'Osijek - Okolica', 'Rijeka', 'Pula', 'Zagreb',
       'Zadar - Okolica', 'Sveti Ivan Zelina', 'Vrbovec', 'Dubrava',
       'Karlovac - Okolica', 'Varaždin', 'Dicmo', 'Hvar', 'Pisarovina',
       'Split', 'Buje', 'Podsused - Vrapče', 'Čakovec - Okolica', 'Darda',
       'Petrinja', 'Solin', 'Krk', 'Maksimir', 'Otočac', 'Brezovica',
       'Sisak', 'Slavonski Brod', 'Požega', 'Duga Resa', 'Bjelovar',
       'Imotski', 'Metković', 'Pakrac', 'Buzet', 'Makarska',
       'Biograd Na Moru', 'Solin 

In [26]:
locations = df_i["Location"].apply(lambda x: unidecode(str(x)).lower())

In [27]:
result = locations.isin(towns_low)
len(result) - result.sum()

6241

I'd have to drop 6241 cars for location. Before I do that I'll try to save some more. I see that I have values such as Zagreb - okolica etc. Those could be saved. 

In [28]:
locs = df_i["Location"].apply(lambda x: str(x).split("-")[0].strip() if len(str(x).split("-")) else x)
locations = locs.apply(lambda x: unidecode(x).lower())
res = locations.isin(towns_low)
len(res) - res.sum()

4820

It's still not perfect, but I decided to drop the cars without useful location info because it's either dropping 4820 cars or the entire analysis based on location and I think that the latter is more valuable.

In [29]:
new_df_i = df_i.copy()
new_df_i['Town'] = pd.Series([])
new_df_i['County'] = pd.Series([])

new_df_i

C:\Users\saind\AppData\Local\Temp\ipykernel_24456\2520334235.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df_i['Town'] = pd.Series([])
C:\Users\saind\AppData\Local\Temp\ipykernel_24456\2520334235.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df_i['County'] = pd.Series([])


,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County
0,Osijek,29.03.2023,18,13600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215000.00,2018,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN,NaN,NaN
1,Labin,29.03.2023,9,7500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304730.00,2011,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,NaN,NaN
2,Sveta Nedelja,29.03.2023,8,7700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157000.00,2012,...,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN,NaN,NaN
3,Črnomerec,29.03.2023,17,2750,Peugeot 207 1.4\r\n,1.4,Benzin + LPG,Uredno,190000.00,2007,...,Prednji,gotovina,Prodajem jer je višak,Mehanički,NaN,NaN,NaN,NaN,NaN,NaN
4,Slatina,29.03.2023,20,5700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385000.00,2009,...,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21901,Osijek,25.06.2017,7517,5700,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN,NaN,NaN
21902,Sisak,08.02.2017,10136,4500,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN,NaN,NaN
21903,Nova cesta,03.02.2017,852,8500,Volvo S80 2.4D\r\n,2.4D,Diesel,Uredno,240000.00,2008,...,Prednji,gotovina,NaN,Automatski,NaN,Volvo,S80,NaN,NaN,NaN
21904,Podsused - Vrapče,01.01.2017,23049,11300,VW Golf V GTI\r\n,GTI,Benzin,Kao novo,146200.00,2008,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,VW,Golf V,NaN,NaN,NaN


In [30]:
pattern_town_low = r"\b{}\b".format(r"\b|\b".join(map(re.escape, towns_low)))

for index, row in new_df_i.iterrows():
    loc = unidecode(str(row['Location'])).lower()
    match = re.search(pattern_town_low, loc)
    match_text = match.group() if match else None
    
    if match_text:
        location = locations_df[locations_df['Town_Low'] == match_text].iloc[0]
        town_text = location['City/Town']
        county_text = location['Zupanija']
        new_df_i.at[index, 'Town'] = town_text
        new_df_i.at[index, 'County'] = county_text

new_df_i

,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County
0,Osijek,29.03.2023,18,13600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215000.00,2018,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN,Osijek,Osječko-baranjska
1,Labin,29.03.2023,9,7500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304730.00,2011,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,Labin,Istarska
2,Sveta Nedelja,29.03.2023,8,7700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157000.00,2012,...,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN,Sveta Nedelja,Zagrebačka
3,Črnomerec,29.03.2023,17,2750,Peugeot 207 1.4\r\n,1.4,Benzin + LPG,Uredno,190000.00,2007,...,Prednji,gotovina,Prodajem jer je višak,Mehanički,NaN,NaN,NaN,NaN,NaN,NaN
4,Slatina,29.03.2023,20,5700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385000.00,2009,...,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN,Slatina,Virovitičko-podravska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21901,Osijek,25.06.2017,7517,5700,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN,Osijek,Osječko-baranjska
21902,Sisak,08.02.2017,10136,4500,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN,Sisak,Sisačko-moslavačka
21903,Nova cesta,03.02.2017,852,8500,Volvo S80 2.4D\r\n,2.4D,Diesel,Uredno,240000.00,2008,...,Prednji,gotovina,NaN,Automatski,NaN,Volvo,S80,NaN,NaN,NaN
21904,Podsused - Vrapče,01.01.2017,23049,11300,VW Golf V GTI\r\n,GTI,Benzin,Kao novo,146200.00,2008,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,VW,Golf V,NaN,NaN,NaN


In [31]:
new_df_i = new_df_i.dropna(subset='Town')

In [32]:
new_df_i[new_df_i['Town'].isnull()]

,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County


In [33]:
new_df_i[new_df_i['County'].isnull()]

,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County


In [34]:
df_i = new_df_i.reset_index(drop=True).copy()

In [35]:
df_i

,Location,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County
0,Osijek,29.03.2023,18,13600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215000.00,2018,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN,Osijek,Osječko-baranjska
1,Labin,29.03.2023,9,7500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304730.00,2011,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,Labin,Istarska
2,Sveta Nedelja,29.03.2023,8,7700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157000.00,2012,...,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN,Sveta Nedelja,Zagrebačka
3,Slatina,29.03.2023,20,5700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385000.00,2009,...,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN,Slatina,Virovitičko-podravska
4,Osijek,29.03.2023,19,2600,Opel Astra 1.7cdti\r\n,1.7cdti,Diesel,Uredno,220000.00,2005,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,Osijek,Osječko-baranjska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18229,Vukovar,20.11.2017,2703,6000,Mercedes CLK coupe 270cdi avantgarde\r\n,270cdi,Diesel,Uredno,241000.00,2005,...,Zadnji,gotovina,Kupovina drugog vozila,Automatski,NaN,Mercedes-Benz,CLK coupe,NaN,Vukovar,Vukovarsko-srijemska
18230,Velika Gorica,03.11.2017,8882,20499,"Audi TT, 2.0TDI, full oprema, quattro, reg.6mj...",2.0TDI,Diesel,Kao novo,155000.00,2013,...,4x4,gotovina,Kupovina drugog vozila,Automatski-sekvencijski,NaN,Audi,TT,NaN,Velika Gorica,Zagrebačka
18231,Osijek,25.06.2017,7517,5700,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN,Osijek,Osječko-baranjska
18232,Sisak,08.02.2017,10136,4500,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255000.00,2007,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN,Sisak,Sisačko-moslavačka


### Deleting Unnecessary Columns

The first thing I can get rid of is the 'Location' column, since I've sorted out Town and County columns.

In [36]:
df_o = df_o.drop('Location', axis=1)
df_o

,Active_Since,Times_Viewed,Price_in_Euros,Proizvođač,Model,Godina_proizvodnje,Kilometraža,Registriran_do,Snaga_motora,Radni_obujam,...,Vlasnik,Airbag,Garažiran,Novo/Rabljeno,Servisna_knjižica,CO2-Emisija,Moguća_zamjena,Oldtimer,Town,County
0,29.03.2023.,4,1.000,VW,Golf IV,1998.00,370000 km,07/2023,74 kW,1600 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
1,29.03.2023.,297,16.500,Citroen,C4 Grand Picasso,2015.00,128400 km,NaN,110 kW,1997 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
2,29.03.2023.,264,31.000,Škoda,Kodiaq,2019.00,63400 km,NaN,110 kW,1968 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3,29.03.2023.,330,24.400,VW,Tiguan,2017.00,113600 km,NaN,110 kW,1968 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
4,29.03.2023.,196,5.500,Dacia,Logan,2015.00,257000 km,04/2023,66 kW,1461 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,08.01.2021.,540,"11.533,61",NaN,NaN,2018.00,164118 km,NaN,81 kW,1598 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pula,Istarska
3395,08.01.2021.,402,17.400,NaN,NaN,2017.00,132210 km,NaN,125 kW,1956 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3396,08.01.2021.,682,21.900,NaN,NaN,2017.00,92908 km,NaN,110 kW,1999 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka
3397,08.01.2021.,634,39.500,NaN,NaN,2017.00,96524 km,NaN,132 kW,1999 cm3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zagreb,Zagrebačka


In [37]:
df_i = df_i.drop('Location', axis=1)
df_i

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Stanje_vozila,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,...,Vrsta_pogona,Načini_plaćanja,Razlog_prodaje,Vrsta_mjenjača,Garancija_za_vozilo,Marka,Model,Radni_obujam_u_cm3,Town,County
0,29.03.2023,18,13600,"VW GOLF 7.5 1.6 TDI,NOVI MODEL,NAVI,REG. GOD. ...",1.6 TDI,Diesel,Kao novo,215000.00,2018,85.00,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,NaN,NaN,NaN,Osijek,Osječko-baranjska
1,29.03.2023,9,7500,Ford S-Max 2.0 tdci\r\n,2.0 tdci,Diesel,Uredno,304730.00,2011,103.00,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,Labin,Istarska
2,29.03.2023,8,7700,"Škoda Octavia 1,6 tdi 105 ks\r\n","1,6 tdi 105 ks",Diesel,Uredno,157000.00,2012,77.00,...,Prednji,gotovina,NaN,Mehanički,2 godine,NaN,NaN,NaN,Sveta Nedelja,Zagrebačka
3,29.03.2023,20,5700,BMW serija 3 Touring 320D LCI REDIZAJN**2009 G...,320 D,Diesel,Uredno,385000.00,2009,130.00,...,Zadnji,gotovina,Prodajem jer je višak,Automatski,NaN,NaN,NaN,NaN,Slatina,Virovitičko-podravska
4,29.03.2023,19,2600,Opel Astra 1.7cdti\r\n,1.7cdti,Diesel,Uredno,220000.00,2005,74.00,...,Prednji,gotovina,NaN,Mehanički,NaN,NaN,NaN,NaN,Osijek,Osječko-baranjska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18229,20.11.2017,2703,6000,Mercedes CLK coupe 270cdi avantgarde\r\n,270cdi,Diesel,Uredno,241000.00,2005,125.00,...,Zadnji,gotovina,Kupovina drugog vozila,Automatski,NaN,Mercedes-Benz,CLK coupe,NaN,Vukovar,Vukovarsko-srijemska
18230,03.11.2017,8882,20499,"Audi TT, 2.0TDI, full oprema, quattro, reg.6mj...",2.0TDI,Diesel,Kao novo,155000.00,2013,125.00,...,4x4,gotovina,Kupovina drugog vozila,Automatski-sekvencijski,NaN,Audi,TT,NaN,Velika Gorica,Zagrebačka
18231,25.06.2017,7517,5700,HITNO Opel Vivaro 1.9 cdti 2007. Produzeni OBN...,1.9 cdti,Diesel,Uredno,525000.00,2007,74.00,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Opel,Vivaro,NaN,Osijek,Osječko-baranjska
18232,08.02.2017,10136,4500,"Honda Civic 2.2 i-cdti, reg. do 12/2023\r\n",2.2 i-cdti,Diesel,Uredno,255000.00,2007,103.00,...,Prednji,gotovina,Kupovina drugog vozila,Mehanički,NaN,Honda,Civic,NaN,Sisak,Sisačko-moslavačka


In [38]:
print(df_o.columns)
print(df_i.columns)

Index(['Active_Since', 'Times_Viewed', 'Price_in_Euros', 'Proizvođač', 'Model',
       'Godina_proizvodnje', 'Kilometraža', 'Registriran_do', 'Snaga_motora',
       'Radni_obujam', 'Vrsta_goriva', 'Stanje_vozila', 'Broj_brzina',
       'Vrsta_mjenjača', 'Oblik_karoserije', 'Broj_vrata', 'Broj_sjedala',
       'Boja', 'Pogon', 'Klima_uređaj', 'Vlasnik', 'Airbag', 'Garažiran',
       'Novo/Rabljeno', 'Servisna_knjižica', 'CO2-Emisija', 'Moguća_zamjena',
       'Oldtimer', 'Town', 'County'],
      dtype='object')
Index(['Active_Since', 'Times_Viewed', 'Price_in_Euros', '_Title', 'Tip',
       'Motor', 'Stanje_vozila', 'Prijeđeni_kilometri', 'Godina_proizvodnje',
       'Snaga_motora_kW', 'Godina_modela', 'Prodavač', 'Registriran_do',
       'Boja_vozila', 'Broj_stupnjeva_na_mjenjaču', 'Broj_vrata',
       'Oblik_karoserije', 'Prosječna_potrošnja_goriva_l/100km',
       'Radni_obujam_cm3', 'Ostali_podaci_o_vozilu', 'Ovjes', 'Starost',
       'Vlasnik', 'Vrsta_pogona', 'Načini_plaćanja', 'R

I'll drop the columns that are not present in both datasets and cannot be found out by the model of the car without further investigation, and also the columns that give no useful information for my analysis.

In [39]:
df_o = df_o.drop(['Servisna_knjižica', 'Moguća_zamjena', 'Stanje_vozila'], axis=1)

In [40]:
df_i = df_i.drop(['Stanje_vozila', 'Načini_plaćanja', 'Razlog_prodaje', 'Garancija_za_vozilo'], axis=1)

I'll investigate some more into columns that I'm not clear about what info is in them and how many records have a valid value.

In [41]:
df_o.isnull().sum()

Active_Since             0
Times_Viewed             0
Price_in_Euros           0
Proizvođač             943
Model                  956
Godina_proizvodnje       5
Kilometraža            858
Registriran_do        2435
Snaga_motora           494
Radni_obujam           576
Vrsta_goriva          3399
Broj_brzina           3399
Vrsta_mjenjača        3399
Oblik_karoserije      3399
Broj_vrata            3399
Broj_sjedala          3399
Boja                  3399
Pogon                 3399
Klima_uređaj          3399
Vlasnik               3399
Airbag                3399
Garažiran             3399
Novo/Rabljeno         3399
CO2-Emisija           3399
Oldtimer              3399
Town                     0
County                   0
dtype: int64

In [42]:
df_o = df_o.drop('Vlasnik', axis=1)

I'll drop these columns for now. I'm considering dropping the entire dataset if the other dataset has all this info on it's cars.

In [43]:
df_o = df_o.drop(['Vrsta_goriva', 'Broj_brzina', 'Vrsta_mjenjača',
       'Oblik_karoserije', 'Broj_vrata', 'Broj_sjedala', 'Boja', 'Pogon',
       'Klima_uređaj', 'Airbag', 'Garažiran', 'Novo/Rabljeno', 'CO2-Emisija',
       'Oldtimer'], axis=1)

In [44]:
df_i['Tip'].unique()

array(['1.6 TDI', '2.0 tdci', '1,6 tdi 105 ks', ..., '16 d',
       '1,6 e-hdi Tepee', '1.1 66ks'], dtype=object)

In [45]:
df_i['Prodavač']

0        privatna osoba
1        privatna osoba
2          pravna osoba
3        privatna osoba
4        privatna osoba
              ...      
18229    privatna osoba
18230    privatna osoba
18231    privatna osoba
18232    privatna osoba
18233    privatna osoba
Name: Prodavač, Length: 18234, dtype: object

In [46]:
df_i.isnull().sum()

Active_Since                              0
Times_Viewed                              0
Price_in_Euros                            0
_Title                                    0
Tip                                    1081
Motor                                     8
Prijeđeni_kilometri                    1005
Godina_proizvodnje                        0
Snaga_motora_kW                        1863
Godina_modela                          1613
Prodavač                                  0
Registriran_do                         2710
Boja_vozila                             333
Broj_stupnjeva_na_mjenjaču              793
Broj_vrata                               65
Oblik_karoserije                        512
Prosječna_potrošnja_goriva_l/100km    12534
Radni_obujam_cm3                       8138
Ostali_podaci_o_vozilu                 9697
Ovjes                                   353
Starost                                  62
Vlasnik                                 725
Vrsta_pogona                    

In [47]:
df_i['Ostali_podaci_o_vozilu']

0              Garažiran
1                    NaN
2              Garažiran
3                Metalik
4                    NaN
              ...       
18229          Garažiran
18230          Garažiran
18231    Servisna knjiga
18232          Garažiran
18233          Garažiran
Name: Ostali_podaci_o_vozilu, Length: 18234, dtype: object

In [48]:
df_i['Vlasnik']

0        Drugi
1        Drugi
2         Prvi
3        Drugi
4        Drugi
         ...  
18229    Drugi
18230    Drugi
18231    Drugi
18232    Drugi
18233     Prvi
Name: Vlasnik, Length: 18234, dtype: object

In [49]:
df_i = df_i.drop(['Prodavač', 'Radni_obujam_u_cm3', 'Ostali_podaci_o_vozilu', 'Starost'], axis=1)

I'm going to give up on the Oglasnik dataset and keep working with the Index data. First, I'll try and salvage some of the cars that are missing brand and model by trying to extract it from the title, as that is the reason I scraped the titles in the first place.

In [50]:
df = df_i.copy()

### Brand

In [51]:
for index, row in df[df['Marka'].isnull()].iterrows():
    title = row['_Title'].split()
    
    df.at[index, 'Marka'] = title[0]

In [52]:
df['Marka'].unique()

array(['VW', 'Ford', 'Škoda', 'BMW', 'Opel', 'Fiat', 'Mercedes-Benz',
       'Renault', 'Audi', 'Mini', 'Citroen', 'Alfa', 'Chevrolet', 'Smart',
       'Peugeot', 'AUDI', 'MINI', 'Volvo', 'WARTBURG', 'Lincoln',
       'HYUNDAI', 'Hyundai', 'Nissan', 'Toyota', 'PEUGEOT', 'Mazda',
       'otkup', 'Hitno', 'Land', 'RENAULT', 'Daewoo', 'Kia', '***KUPUJEM',
       'Lancia', 'VOLVO', 'Subaru', 'Seat', 'Suzuki', 'Alfa Romeo',
       'Chrysler', 'Dacia', 'Jeep', 'Honda', 'Rover', 'FIAT', 'Citreon',
       'Mitsubishi', 'Mercedes', 'Grand', 'Volkswagen', 'Porsche',
       '2012.', 'MAZDA', 'Daihatsu', 'Saab', 'Jaguar', 'Bmw', 'Maserati',
       'Yugo', 'Infiniti', 'Isuzu', 'Land Rover', 'Lada', 'Ssang Yong',
       'Lexus', 'DS Automobiles', 'Polo', 'Tesla', 'Hummer', 'Zastava',
       'Wartburg', 'Ssang', 'MG', 'Pontiac', 'Dodge', 'VOLKSWAGEN',
       'Cadillac', 'Cupra', 'UAZ', 'OPEL', 'Prodajem', 'Oglas'],
      dtype=object)

In [53]:
not_brands = ['otkup', 'Hitno', '***KUPUJEM', '2012.', 'Prodajem', 'Oglas', 'Grand']
mask = df['Marka'].isin(not_brands)
df = df[~mask]

In [54]:
df[df['Marka'] == 'Ssang']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
6881,10.02.2023,477,5500,Ssang Yong Actyon 2.0 xDI\r\n,2.0 xDI,Diesel,164153.00,2007,104.00,2007.00,...,NaN,2.0,Obični,Drugi,Zadnji,Mehanički,Ssang,NaN,Karlovac,Karlovačka


I can remember these car and manually set it's brand and model, since it's just one car. It has all the other info, so why drop it?

In [55]:
df[df['Marka'] == 'Land']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
146,29.03.2023,102,13300,Land Rover Freelander Land Rover 3.0 V6 Specia...,"Land Rover 3.0 V6 Special Edition 320ks,bi-xen...",Diesel,NaN,2009,180.00,2009.00,...,NaN,NaN,Obični,Drugi,Prednji,Automatski,Land,NaN,Virovitica,Virovitičko-podravska


This one will get dropped later on because of the many missing values, so I won't bother with it.

In [56]:
df = df.drop(df[df['Marka'] == 'Land'].index)

In [57]:
df = df.drop(df[df['Marka'] == 'Grand'].index)

In [58]:
df[df['Marka'] == 'Rover']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
454,28.03.2023,157,7000,Rover 220 SBTurbo\r\n,SBTurbo,Benzin,212000.00,1995,147.00,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Rover,NaN,Velika Gorica,Zagrebačka
2174,20.03.2023,273,2600,Rover 75 2.0 CDT**180 xxx KM**ISPIS **RWG GOD ...,2.0 CDT,Diesel,NaN,2000,85.00,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Rover,75,Dubrava,Zagrebačka
2599,18.03.2023,379,7000,Rover 220 BTurbo targa coupe\r\n,BTurbo targa coupe,Benzin,212000.00,1995,168.00,2003.00,...,NaN,NaN,Sportski,Drugi,Prednji,Mehanički,Rover,220,Velika Gorica,Zagrebačka
3246,14.03.2023,208,2400,Rover 75 2.0 cdt\r\n,2.0 cdt,Diesel,240000.00,2001,85.00,2001.00,...,6.00,2.0,Obični,Drugi,Prednji,Mehanički,Rover,75,Bjelovar,Bjelovarsko-bilogorska
8476,20.01.2023,627,1700,Rover 25 1.4 16V\r\n,1.4 16V,Benzin,150000.00,2000,77.00,2000.00,...,NaN,1.400,Obični,Drugi,Prednji,Mehanički,Rover,25,Tučepi,Splitsko-dalmatinska
15908,19.06.2022,3978,1063,"Rover 75 2000 , 2004 g. Zamjena za skuter ili ...",2000,Diesel,260000.00,2004,120.00,2000.00,...,6.00,2.00,Obični,Drugi,Prednji,Mehanički,Rover,75,Varaždin,Varaždinska
17895,18.06.2021,1438,2400,Rover 25 TD\r\n,TD,Diesel,1.00,2001,NaN,2000.00,...,5.00,2.00,Obični,Drugi,Prednji,Mehanički,Rover,25,Kaštela,Splitsko-dalmatinska
18178,30.11.2019,3823,3000,Rover 620 Sdi redistriran novi lak\r\n,Sdi,Diesel,NaN,1998,NaN,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Rover,620,Velika Gorica,Zagrebačka


In [59]:
df[df['Marka'] == 'Polo']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
3497,13.03.2023,353,9200,"Polo 1,4 TDI BMT Comfortline\r\n\t\t",1.4 TDI,Diesel,124000.00,2017,55.00,2015.00,...,NaN,1.4,Obični,Drugi,Prednji,Mehanički,Polo,NaN,Rijeka,Primorsko-goranska


In [60]:
for index, row in df.iterrows():
    brand = row['Marka']
    if(brand.isupper() and brand not in ['UAZ', 'BMW', 'MG', 'VW', 'MINI']):
        df.at[index, 'Marka'] = brand.capitalize()
    elif(brand == 'Bmw'):
        df.at[index, 'Marka'] = 'BMW'
    elif(brand == 'Citreon'):
        df.at[index, 'Marka'] = 'Citroen'
    elif(brand == 'VW'):
        df.at[index, 'Marka'] = 'Volkswagen'
    elif(brand == 'Ssang'):
        df.at[index, 'Marka'] = 'Ssang Yong'
    elif(brand == 'Alfa'):
        df.at[index, 'Marka'] = 'Alfa Romeo'
    elif(brand == 'Mercedes'):
        df.at[index, 'Marka'] = 'Mercedes-Benz'
    elif(brand == 'Polo'):
        df.at[index, 'Marka'] = 'Volkswagen'
        df.at[index, 'Model'] = 'Polo'
    elif(brand == 'Mini'):
        df.at[index, 'Marka'] = 'MINI'
        

In [61]:
df['Marka'].unique()

array(['Volkswagen', 'Ford', 'Škoda', 'BMW', 'Opel', 'Fiat',
       'Mercedes-Benz', 'Renault', 'Audi', 'MINI', 'Citroen',
       'Alfa Romeo', 'Chevrolet', 'Smart', 'Peugeot', 'Volvo', 'Wartburg',
       'Lincoln', 'Hyundai', 'Nissan', 'Toyota', 'Mazda', 'Daewoo', 'Kia',
       'Lancia', 'Subaru', 'Seat', 'Suzuki', 'Chrysler', 'Dacia', 'Jeep',
       'Honda', 'Rover', 'Mitsubishi', 'Porsche', 'Daihatsu', 'Saab',
       'Jaguar', 'Maserati', 'Yugo', 'Infiniti', 'Isuzu', 'Land Rover',
       'Lada', 'Ssang Yong', 'Lexus', 'DS Automobiles', 'Tesla', 'Hummer',
       'Zastava', 'MG', 'Pontiac', 'Dodge', 'Cadillac', 'Cupra', 'UAZ'],
      dtype=object)

### Model

Now that that's sorted out, let's look at the models.

In [62]:
added_models = []
for index, row in df[df['Model'].isnull()].iterrows():
    title = row['_Title'].split()
    brand_words = len(row['Marka'].split())
    brand = None
    if brand_words > 1:
        brand = title[2]
        df.at[index, 'Model'] = brand
    else:
        if title[1].lower() == 'serija':
            brand = title[1] + ' ' + title[2]
            df.at[index, 'Model'] = brand
        else:
            brand = title[1]
            df.at[index, 'Model'] = brand
    added_models.append(brand)
    

In [63]:
set(added_models)

{'100',
 '1007',
 '118',
 '123',
 '124',
 '146',
 '147',
 '156',
 '159',
 '166',
 '180',
 '1802',
 '2',
 '2008',
 '206',
 '206//KLIMA,DALJINSKO,PODIZACI//REG',
 '207',
 '208',
 '220',
 '3',
 '3008',
 '301',
 '307',
 '308',
 '318',
 '353',
 '4',
 '420d',
 '500',
 '5008',
 '508',
 '520d',
 '6',
 '607',
 '850',
 '9,3',
 'A',
 'A-klasa',
 'A1',
 'A3',
 'A4',
 'A5',
 'A6',
 'Accent',
 'Accord',
 'Actyon',
 'Almera',
 'Altea',
 'Arona',
 'Astra',
 'Auris',
 'Aveo',
 'Aztek',
 'B-2500',
 'B-klasa',
 'BRAVO',
 'BUBA',
 'Beetle',
 'Benz',
 'Berlingo',
 'Boxer',
 'Bravo',
 'Buba',
 'C',
 'C-Max',
 'C-klasa',
 'C2',
 'C3',
 'C4',
 'C5',
 'CAPTUR',
 'CL',
 'CLC',
 'CLK',
 'CLS',
 'CORSA',
 'CR-V',
 'CX-3',
 'CX-5',
 'Caddy',
 'Captiva',
 'Captur',
 'Cayenne',
 'Ceed',
 'Celica',
 'Cherokee',
 'Citan',
 'Civic',
 'Clio',
 'Colt',
 'Combo',
 'Cooper',
 'Corolla',
 'Corsa',
 'Coupe',
 'Crossland',
 'DS',
 'Delta',
 'Doblo',
 'Ducato',
 'Duster',
 'E',
 'E-klasa',
 'E36',
 'E90',
 'E93',
 'Eos',
 'F10

In [64]:
df[df['Model'] == '2']['Marka'].unique()

array(['Mazda'], dtype=object)

In [65]:
df[df['Model'] == '206//KLIMA,DALJINSKO,PODIZACI//REG']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
105,29.03.2023,53,1400,"PEUGEOT 206//KLIMA,DALJINSKO,PODIZACI//REG 11M...",NaN,Benzin,223000.00,2005,NaN,NaN,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Peugeot,"206//KLIMA,DALJINSKO,PODIZACI//REG",Slavonski Brod,Brodsko-posavska


In [66]:
df = df.drop(df[df['Model'] == '206//KLIMA,DALJINSKO,PODIZACI//REG'].index)

In [67]:
df[df['Model'] == '3']['Marka'].unique()

array(['Mazda', 'BMW'], dtype=object)

In [68]:
df.loc[(df['Model'] == '3') & (df['Marka'] == 'BMW'), 'Model'] = 'serija 3'

In [69]:
df[df['Model'] == '4']['Marka'].unique()

array(['Renault'], dtype=object)

In [70]:
df[df['Model'] == '6']['Marka'].unique()

array(['Mazda'], dtype=object)

In [71]:
df[df['Model'] == '9,3']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
1244,25.03.2023,329,6000,"Karavan Saab 9,3 1.9 TiD SportCombi Vector aut...",1.9 TiD SportCombi Vector,Diesel,245400.00,2008,110.00,2008.00,...,NaN,1.91,Sportski,Treći i više,Prednji,Automatski,Saab,"9,3",Zadar,Zadarska
7750,30.01.2023,1797,4000,"Saab 9,3 2.8 TS Aero\r\n",2.8 TS Aero,Benzin,340000.00,2006,184.00,2006.00,...,11.00,2.8,Sportski,Drugi,Prednji,Mehanički,Saab,"9,3",Samobor,Zagrebačka
8378,21.01.2023,1417,3500,"Saab 9,3 2.0 turbo\r\n",2.0 turbo,Benzin,200000.00,2002,110.00,2000.00,...,NaN,2.000,Obični,Drugi,Prednji,Mehanički,Saab,"9,3",Vodice,Šibensko-kninska
8743,17.01.2023,768,3500,"Saab 9,3 1.8 I\r\n",1.8 I,Benzin,200000.00,2006,90.00,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Saab,"9,3",Zadar,Zadarska
12840,06.10.2022,1782,3500,"Saab 9,3 2.0 Turbo\r\n",2.0 Turbo,Benzin,NaN,2002,110.00,2002.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Saab,"9,3",Virovitica,Virovitičko-podravska
13429,18.09.2022,1445,3900,Saab 9.3 2.2 tid\r\n,2.2 tid,Diesel,320000.00,2004,90.00,2004.00,...,NaN,2.2,Obični,Drugi,Prednji,Mehanički,Saab,"9,3",Varaždin,Varaždinska
14292,21.08.2022,1607,6300,"Saab 9,3 1.9 vector\r\n",1.9 vector,Diesel,217000.00,2007,110.00,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Automatski,Saab,"9,3",Zagreb,Zagrebačka
14672,04.06.2022,3281,4000,"Saab 9,3 2.0T stg2\r\n",2.0T,Benzin,270000.00,2004,NaN,2003.00,...,NaN,2.00,Sportski,NaN,Prednji,Mehanički,Saab,"9,3",Bilje,Osječko-baranjska
15002,25.07.2022,1057,2000,"Saab 9,3 2,0 turbo\r\n\t\t","2,0 turbo",Benzin,NaN,2002,NaN,2000.00,...,NaN,2.00,Obični,Drugi,Prednji,Mehanički,Saab,"9,3",Šibenik,Šibensko-kninska
16480,23.05.2022,3851,3600,"Saab 9,3 Aero 2.0 T benzin 210KS\r\n",Aero 2.0 T benzin 210KS,Benzin,283000.00,2003,154.00,NaN,...,NaN,NaN,Sportski,Treći i više,Prednji,Mehanički,Saab,"9,3",Osijek,Osječko-baranjska


In [72]:
df[df['Model'] == 'A']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
5471,24.02.2023,234,37000,Mercedes-Benz A 200d Progressive Limousine\r\n,200d Progressive Limousine,Diesel,20770.00,2022,110.00,2022.00,...,NaN,1.95,Obični,Prvi,Prednji,Automatski,Mercedes-Benz,A,Velika Gorica,Zagrebačka
5474,24.02.2023,199,33000,Mercedes-Benz A 200d Progressive\r\n,200d Progressive,Diesel,47860.00,2021,110.00,2021.00,...,NaN,1.95,Obični,Prvi,Prednji,Automatski,Mercedes-Benz,A,Velika Gorica,Zagrebačka


In [73]:
df.loc[df['Model'] == 'A', 'Model'] = 'A-klasa'

In [74]:
df.loc[df['Model'] == 'Benz', 'Model'] = 'GLA'

In [75]:
df[df['Model'] == 'C']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
52,29.03.2023,53,10500,Mercedes-Benz C 180CDI T\r\n,180CDI,Diesel,180000.00,2011,88.00,2011.00,...,6.00,2.2,Obični,Drugi,Zadnji,Mehanički,Mercedes-Benz,C,Zagreb,Zagrebačka
56,29.03.2023,43,9900,Mercedes-Benz C 180CDI T\r\n,180CDI,Diesel,230000.00,2011,88.00,2011.00,...,6.00,2.2,Obični,Drugi,Zadnji,Mehanički,Mercedes-Benz,C,Zagreb,Zagrebačka
2670,18.03.2023,865,11000,"Mercedes C klasa 220 cdi 7G Tronic, Avantgarde...",220 CDI 7G Tronic,Diesel,390000.00,2011,125.00,2011.00,...,NaN,2.143,Obični,Drugi,Zadnji,Automatski,Mercedes-Benz,C,Split,Splitsko-dalmatinska


In [76]:
df.loc[df['Model'] == 'C'] = 'C-klasa'

In [77]:
df[df['Model'] == 'Coupe']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
81,29.03.2023,41,2000,"Hyundai Coupe 1.6, reg. god dana\r\n","1.6, reg. god dana",Benzin,334000.00,2002,77.00,2002.00,...,NaN,1.599,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Zagreb,Zagrebačka
1051,26.03.2023,282,2500,Hyundai Coupe 1.6 16v\r\n,1.6 16v,Benzin,480000.00,2002,77.00,2002.00,...,NaN,NaN,Sportski,Drugi,Prednji,Mehanički,Hyundai,Coupe,Bjelovar,Bjelovarsko-bilogorska
2858,16.03.2023,612,1000,Hyundai Coupe 1.6 16v\r\n,1.6 16v,Benzin,202000.00,2000,76.00,2000.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Split,Splitsko-dalmatinska
2985,16.03.2023,474,1500,Hyundai Coupe 1.6 benzin\r\n,1.6 benzin,Benzin,203000.00,1998,84.00,1998.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Zabok,Krapinsko-zagorska
3775,11.03.2023,286,2350,Hyundai Coupe 1.6-atestirani plin-ODLIČNO STAN...,1.6,Benzin,350000.00,1997,83.00,1997.00,...,7.00,1.6,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Vrbovec,Zagrebačka
7955,27.01.2023,783,2500,"Hyundai Coupe 1.6, LPG, 2003\r\n","1.6, LPG, 2003",Benzin,339000.00,2003,77.00,2003.00,...,NaN,16.0,Obični,Treći i više,Prednji,Mehanički,Hyundai,Coupe,Zagreb,Zagrebačka
8923,14.01.2023,971,7000,Hyundai Coupe fx\r\n,fx,Benzin,200000.00,2008,105.00,2008.00,...,8.00,1.975,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Vinkovci,Vukovarsko-srijemska
8961,14.01.2023,1281,3999,Hyundai Coupe 2.7 V6 GK 2003g\r\n,2.7 V6 GK,Benzin,250000.00,2003,124.00,2003.00,...,9.00,2.700,Obični,Drugi,Prednji,Mehanički,Hyundai,Coupe,Vinkovci,Vukovarsko-srijemska
9537,04.01.2023,994,5900,Hyundai Coupe 2.0\r\n,2.0,Benzin,152000.00,2004,102.00,2004.00,...,NaN,NaN,Sportski,Treći i više,Prednji,Mehanički,Hyundai,Coupe,Zadar,Zadarska
9906,26.12.2022,1685,6100,Hyundai Coupe 2.7 DOHC GLS\r\n,2.7 DOHC GLS,Benzin,197647.00,2008,123.00,2008.00,...,6.00,2.656,Sportski,Drugi,Prednji,Automatski,Hyundai,Coupe,Pula,Istarska


In [78]:
df[df['Model'] == 'E']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
8581,18.01.2023,753,21500,Mercedes-Benz E 200 CDI AMG\r\n,E 200 CDI AMG,Diesel,84800.00,2011,100.00,2011.00,...,NaN,NaN,Sportski,Drugi,Zadnji,Automatski,Mercedes-Benz,E,Zagreb,Zagrebačka


In [79]:
df.loc[df['Model'] == 'E', 'Model'] = 'E-klasa'

In [80]:
df[df['Model'] == 'Grand']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
868,26.03.2023,134,15750,"Renault Grand Scenic 1,5 dCi 7 Sjedala,navigac...","1,5 dCi",Diesel,177000.00,2016,81.00,2017.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Renault,Grand,Oprisavci,Brodsko-posavska
2893,16.03.2023,463,20000,Jeep Grand Cherokee 3.0 CRD\r\n,3.0 CRD,Diesel,280000.00,2011,177.00,2012.00,...,NaN,NaN,Zračni,Drugi,4x4,Automatski,Jeep,Grand,Duga Resa,Karlovačka
4968,01.03.2023,922,7900,Suzuki Grand Vitara 4x4 1.6 PLIN\r\n,1.6,Benzin,170000.00,2007,NaN,NaN,...,NaN,NaN,Obični,Drugi,4x4,Mehanički,Suzuki,Grand,Donja Dubrava,Međimurska
16081,10.06.2022,3862,3500,Jeep Grand Cherokee 5.2 V8\r\n\t\t,5.2 V8,Benzin,350843.00,1993,155.00,1993.00,...,NaN,5.22,Obični,Drugi,Prednji,Mehanički,Jeep,Grand,Slavonski Brod,Brodsko-posavska


In [81]:
for index, row in df[df['Model'] == 'Grand'].iterrows():
    brand = row['Marka']
    new_model = None
    if brand == 'Renault':
        new_model = 'Grand Scenic'
    elif brand == 'Suzuki':
        new_model = 'Grand Vitara'
    else:
        new_model = 'Grand Cherokee'
    df.at[index, 'Model'] = new_model

In [82]:
df[df['Model'] == 'Grande']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
244,28.03.2023,78,3650,"Fiat Grande Punto 1,4 8V\r\n","1,4 8V",Benzin,196000.00,2008,NaN,2008.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Fiat,Grande,Labin,Istarska
469,28.03.2023,126,2400,"Fiat Grande Punto 1,3 jtd 2006g- reg .25.03.20...","Fiat Grande Punto 1,3 jtd",Diesel,260000.00,2006,55.00,2006.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Fiat,Grande,Rijeka,Primorsko-goranska
1539,24.03.2023,229,4200,Fiat Grande Punto 1.4 8v\r\n,1.4 8v,Benzin,165000.00,2011,57.00,2006.00,...,NaN,1.368,Obični,Drugi,Prednji,Mehanički,Fiat,Grande,Dubrava,Zagrebačka
2494,19.03.2023,294,2199,Fiat Grande Punto 1.3 multijet\r\n,1.3 multijet,Diesel,201000.00,2005,57.00,2005.00,...,NaN,NaN,Obični,Drugi,Prednji,Mehanički,Fiat,Grande,Rijeka,Primorsko-goranska


In [83]:
df.loc[df['Model'] == 'Grande', 'Model'] = 'Grande Punto'

In [84]:
df[df['Model'] == 'RS']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
3928,09.03.2023,3309,97000,"Audi RS Q3 Sportback 2,5 TFSI automatik\r\n",NaN,Benzin,8500.00,2022,294.00,2021.00,...,NaN,NaN,Sportski,Prvi,4x4,Automatski,Audi,RS,Zagreb,Zagrebačka


In [85]:
df.loc[df['Model'] == 'RS', 'Model'] = 'RS Q3'

In [86]:
df[df['Model'] == 'S']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
26,29.03.2023,34,2400,"Mercedes-Benz S klasa 320 s plinom, manual mje...","320 s plinom, manual mjenjač",Benzin + LPG,400000.00,1996,170.00,1994.00,...,10.00,3.198,Obični,Drugi,Zadnji,Mehanički,Mercedes-Benz,S,Zagreb,Zagrebačka
2632,18.03.2023,1138,14000,Mercedes-Benz S klasa\r\n,NaN,Diesel,380000.00,2007,173.00,2007.00,...,NaN,2.987,Sportski,Drugi,4x4,Automatski-sekvencijski,Mercedes-Benz,S,Zadar,Zadarska
4094,08.03.2023,921,3499,Mercedes-Benz S klasa\r\n,NaN,Diesel,230000.00,2000,145.00,2000.00,...,8.00,3.0,Zračni,Drugi,Zadnji,Automatski,Mercedes-Benz,S,Pašman,Zadarska


In [87]:
df.loc[df['Model'] == 'S', 'Model'] = 'S-klasa'

In [88]:
df.loc[df['Model'] == 'X3__2.0d__AUTOMAT__KOŽA__PARK', 'Model'] = 'X3' 

In [89]:
df[df['Model'] == 'serija 735i']

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
10561,06.12.2022,5090,7500,BMW serija 735i\r\n,3.5 i,Benzin,340000.00,1998,175.00,1998.00,...,15.00,3.498,Sportski,Treći i više,Zadnji,Automatski,BMW,serija 735i,Split,Splitsko-dalmatinska


In [90]:
df.loc[df['Model'] == 'serija 735i', 'Model'] = 'serija 7'

I will do some more cleaning on the model column, to sort out case (e.g. I obviously don't want 'PASSAT' and 'Passat' to be two different cars), English vs. Croatian names etc.

In [91]:
'BRAVO', 'BUBA', 'Buba', 'CAPTUR', 'CORSA', 'GOLF', 'PASSAT', 'T-CROSS', 'TUCSON', 'a4', 'e46', 'e60', 'f10', 'x2'

('BRAVO',
 'BUBA',
 'Buba',
 'CAPTUR',
 'CORSA',
 'GOLF',
 'PASSAT',
 'T-CROSS',
 'TUCSON',
 'a4',
 'e46',
 'e60',
 'f10',
 'x2')

In [92]:
for index, row in df.iterrows():
    model = row['Model']
    if model.isupper():
        df.at[index, 'Model'] = model.capitalize()
    if model == 'Buba':
        df.at[index, 'Model'] = 'Beetle'
    if model in ['a4', 'e46', 'e60', 'f10', 'x2']:
        df.at[index, 'Model'] = model.capitalize()
    if 'serija' in model:
        df.at[index, 'Model'] = model.replace('serija', 'series')
    if 'klasa' in model:
        df.at[index, 'Model'] = model.replace('klasa', 'class')

In [93]:
set(df['Model'].unique())

{'10',
 '100',
 '100 Avant',
 '1007',
 '100nx',
 '101',
 '106',
 '107',
 '108',
 '11',
 '116',
 '118',
 '123',
 '124',
 '126',
 '126 coupe',
 '127',
 '1300',
 '131',
 '146',
 '147',
 '156',
 '156 sw',
 '159',
 '159 sw',
 '164',
 '166',
 '180',
 '1802',
 '19 Chamade',
 '190',
 '2',
 '2002',
 '2008',
 '204',
 '205',
 '206',
 '206 Break',
 '206 cc',
 '207',
 '207 cc',
 '207 sw',
 '208',
 '220',
 '240',
 '25',
 '3',
 '3008',
 '300c',
 '300m',
 '301',
 '304',
 '306',
 '306 Cabrio',
 '307',
 '307 Break',
 '307 cc',
 '307 sw',
 '308',
 '308 cc',
 '308 sw',
 '311',
 '318',
 '323',
 '323 Sedan',
 '323 f',
 '33',
 '340',
 '353',
 '370z',
 '4',
 '4007',
 '406',
 '406 Break',
 '406 Coupe',
 '407',
 '407 Coupe',
 '420d',
 '45',
 '469',
 '4Runner',
 '5',
 '500',
 '5008',
 '500l',
 '500x',
 '508',
 '520d',
 '55',
 '6',
 '6 Sport Combi',
 '607',
 '620',
 '626',
 '626 Wagon',
 '75',
 '750',
 '760',
 '80',
 '80 Avant',
 '807',
 '850',
 '9,3',
 '9,5',
 '924',
 '940',
 '960',
 'A-class',
 'A1',
 'A2',
 'A

I'm not going to go any further with cleaning up the models, especially since I don't want to decide right now how strict I want to be with the models (e.g. do I need 'A3', 'A3 Cabrio', 'A3 Coupe', 'A3 Sedan', 'A3 Sportback' or can I treat all of them as simply 'A3'). I want it all saved into my database and then I'll see where my analysis leads me and group accordingly. 

### Price

In [94]:
df['Price_in_Euros'].describe()

count     18224
unique     1874
top        2500
freq        301
Name: Price_in_Euros, dtype: int64

In [95]:
df['Price_in_Euros'] = pd.to_numeric(df['Price_in_Euros'], errors='coerce')

In [96]:
df['Price_in_Euros'].max()

630000.0

In [97]:
df[df['Price_in_Euros'].isnull()]

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Godina_modela,...,Prosječna_potrošnja_goriva_l/100km,Radni_obujam_cm3,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
52,C-klasa,C-klasa,NaN,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,...,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-class,C-klasa,C-klasa
56,C-klasa,C-klasa,NaN,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,...,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-class,C-klasa,C-klasa
2670,C-klasa,C-klasa,NaN,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,...,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-klasa,C-class,C-klasa,C-klasa


In [98]:
df = df.dropna(subset='Price_in_Euros')

### Other Null Values

There is nothing I can do about the records that have missing mileage. I can't find that out from other info about the car, and also I need that column for my analysis as mileage is pretty important characteristic of a used car.

In [99]:
df = df.dropna(subset='Prijeđeni_kilometri')

In [100]:
df = df.drop(['Registriran_do', 'Godina_modela'] , axis=1)

In [101]:
df.isnull().sum()

Active_Since                              0
Times_Viewed                              0
Price_in_Euros                            0
_Title                                    0
Tip                                     942
Motor                                     6
Prijeđeni_kilometri                       0
Godina_proizvodnje                        0
Snaga_motora_kW                        1338
Boja_vozila                             300
Broj_stupnjeva_na_mjenjaču              775
Broj_vrata                               62
Oblik_karoserije                        487
Prosječna_potrošnja_goriva_l/100km    11637
Radni_obujam_cm3                       7345
Ovjes                                   338
Vlasnik                                 700
Vrsta_pogona                            150
Vrsta_mjenjača                           49
Marka                                     0
Model                                     0
Town                                      0
County                          

I decided to 'sacrifice' the engine displacement column to save more records.

In [102]:
df = df.drop('Radni_obujam_cm3', axis=1)

In [103]:
df[df['Motor'].isnull()]

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Boja_vozila,...,Oblik_karoserije,Prosječna_potrošnja_goriva_l/100km,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
163,29.03.2023,68,11600.00,"RENAULT CAPTUR TCe 90 LED SVJETLA, NAVIGACIJA,...",90 TCe,NaN,82703.00,2016,66.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Renault,Captur,Osijek,Osječko-baranjska
364,28.03.2023,90,24600.00,"AUDI Q3 2,0 TDI QUATTRO, AUTOMATIK, XENON, PAR...",NaN,NaN,103567.00,2018,110.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Audi,Q3,Osijek,Osječko-baranjska
436,28.03.2023,144,1999.00,Ford Fiesta 1.6 tdi. ZAMJENA ZA VECE ILI KOMB...,1.6 tdi,NaN,320000.00,2005,66.00,NaN,...,Limuzina,4.00,Obični,Drugi,Prednji,Mehanički,Ford,Fiesta,Solin,Splitsko-dalmatinska
5629,23.02.2023,775,2200.00,Citroen Saxo 1.4 vts\r\n,1.4 vts,NaN,220000.00,2000,80.00,NaN,...,NaN,NaN,Obični,Drugi,NaN,Mehanički,Citroen,Saxo,Rijeka,Primorsko-goranska
15537,04.07.2022,3163,7000.00,BMW serija 3 Cabriolet 320CI\r\n,320CI,NaN,270000.00,2001,125.00,Srebrna,...,Limuzina,NaN,Obični,Drugi,Zadnji,Mehanički,BMW,series 3 Cabriolet,Đakovo,Osječko-baranjska
17235,18.04.2022,3587,3307.00,Peugeot Partner\r\n,NaN,NaN,140000.00,2006,55.00,NaN,...,NaN,NaN,NaN,Prvi,NaN,Mehanički,Peugeot,Partner,Pula,Istarska


In [104]:
for index, row in df[df['Motor'].isnull()].iterrows():
    motor = None
    tip = row['Tip']
    title = row['_Title']
    
    if tip == '90 TCe' or tip == '1.4 vts' or tip == '320CI':
        motor = 'Benzin'
    if tip == '1.6 tdi' or 'TDI' in title:
        motor = 'Diesel'
        
    df.at[index, 'Motor'] = motor

In [105]:
df = df.dropna(subset='Motor')

In [106]:
df[df['Tip'].isnull()]

,Active_Since,Times_Viewed,Price_in_Euros,_Title,Tip,Motor,Prijeđeni_kilometri,Godina_proizvodnje,Snaga_motora_kW,Boja_vozila,...,Oblik_karoserije,Prosječna_potrošnja_goriva_l/100km,Ovjes,Vlasnik,Vrsta_pogona,Vrsta_mjenjača,Marka,Model,Town,County
11,29.03.2023,30,5100.00,"BMW 118d reg10/23 Xenon, Alu 17 *5100€*\r\n",NaN,Diesel,322000.00,2007,105.00,Crna,...,Hatchback,NaN,Obični,Drugi,Zadnji,Mehanički,BMW,series 1,Osijek,Osječko-baranjska
14,29.03.2023,35,10700.00,Renault Captur 0.9TCe 2013g - reg 2./2024g\r\n,NaN,Benzin,160000.00,2013,66.00,Narančasta,...,Limuzina,5.00,Obični,NaN,Prednji,Mehanički,Renault,Captur,Našice,Osječko-baranjska
43,29.03.2023,53,6200.00,VW Golf VI 1.6 TDI Bluemotion # 2011 GOD # REG...,NaN,Diesel,290000.00,2011,77.00,Srebrna,...,Karavan,NaN,Obični,Drugi,Prednji,Mehanički,Volkswagen,Golf VI,Vrbovec,Zagrebačka
54,29.03.2023,36,22500.00,BMW X1\r\n,NaN,Diesel,130000.00,2016,140.00,Crna,...,Terensko vozilo (SUV),6.00,Obični,Drugi,4x4,Automatski,BMW,X1,Osijek,Osječko-baranjska
103,29.03.2023,57,8200.00,BMW X3__2.0d__AUTOMAT__KOŽA__PARK SENZORI__TEM...,NaN,Diesel,175000.00,2008,130.00,Plava,...,Terensko vozilo (SUV),NaN,Obični,Drugi,Zadnji,Automatski-sekvencijski,BMW,X3,Slavonski Brod,Brodsko-posavska
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18107,18.06.2020,1372,5973.00,MINI One\r\n,NaN,Benzin,98000.00,2008,70.00,Crna,...,Coupe,8.00,Obični,Treći i više,Prednji,Mehanički,MINI,One,Zagreb,Zagrebačka
18168,22.12.2019,3231,3300.00,VW Golf II dizel - u super stanju\r\n,NaN,Diesel,186000.00,1986,40.00,Crvena,...,Limuzina,NaN,Obični,Prvi,Prednji,Mehanički,Volkswagen,Golf II,Rijeka,Primorsko-goranska
18179,25.11.2019,907,1000.00,Passat B7 djelovi\r\n,NaN,Diesel,260000.00,2014,103.00,Bijela,...,Karavan,NaN,Obični,Drugi,Prednji,Automatski,Volkswagen,Passat Variant,Velika Gorica,Zagrebačka
18184,24.10.2019,944,1000.00,VW Passat Variant\r\n,NaN,Diesel,200000.00,2006,104.00,Bijela,...,Limuzina,NaN,Obični,Drugi,Prednji,Mehanički,Volkswagen,Passat Variant,Velika Gorica,Zagrebačka


In [107]:
df['Tip'].unique()

array(['1.6 TDI', '2.0 tdci', '1,6 tdi 105 ks', ..., '16 d',
       '1,6 e-hdi Tepee', '1.1 66ks'], dtype=object)

This column 'Tip', apart from many missing values, has too many messy values. 

In [108]:
df = df.drop('Tip', axis=1)

Another value I'm willing to sacrifice is 'ovjes'.

In [109]:
df = df.drop('Ovjes', axis=1)

This is the state of the dataset now.

In [110]:
df.isnull().sum()

Active_Since                              0
Times_Viewed                              0
Price_in_Euros                            0
_Title                                    0
Motor                                     0
Prijeđeni_kilometri                       0
Godina_proizvodnje                        0
Snaga_motora_kW                        1338
Boja_vozila                             299
Broj_stupnjeva_na_mjenjaču              775
Broj_vrata                               61
Oblik_karoserije                        486
Prosječna_potrošnja_goriva_l/100km    11636
Vlasnik                                 700
Vrsta_pogona                            149
Vrsta_mjenjača                           49
Marka                                     0
Model                                     0
Town                                      0
County                                    0
dtype: int64

I'll drop all the rows with na values except for null values in l/100km column. That's just too many rows to drop but at the same time I want to keep this column too. So, later on when I'm doing the analysis, I'll do anything related to fuel economy in a separate analysis on the subset of this data that obviously has the needed value.

In [111]:
df = df.dropna(subset=['Snaga_motora_kW', 'Boja_vozila', 'Broj_stupnjeva_na_mjenjaču', 'Broj_vrata', 'Oblik_karoserije', 'Vlasnik', 'Vrsta_pogona', 'Vrsta_mjenjača'])

In [112]:
df.isnull().sum()

Active_Since                             0
Times_Viewed                             0
Price_in_Euros                           0
_Title                                   0
Motor                                    0
Prijeđeni_kilometri                      0
Godina_proizvodnje                       0
Snaga_motora_kW                          0
Boja_vozila                              0
Broj_stupnjeva_na_mjenjaču               0
Broj_vrata                               0
Oblik_karoserije                         0
Prosječna_potrošnja_goriva_l/100km    9080
Vlasnik                                  0
Vrsta_pogona                             0
Vrsta_mjenjača                           0
Marka                                    0
Model                                    0
Town                                     0
County                                   0
dtype: int64

In [113]:
df['Godina_proizvodnje'].unique()

array([2018, 2011, 2012, 2009, 2016, 1992, 2006, 2007, 2005, 2013, 2019,
       2002, 2001, 2003, 2008, 1996, 2010, 2014, 2004, 1999, 2000, 2017,
       1998, 2015, 1987, 2020, 20, 1994, 1997, 2021, 1995, 202, 2022,
       1984, 1990, 1993, 1991, 1970, 1966, 206, 207, 1988, 1982, 1989,
       1986, 1985, 200, 1978, 1983, 1972, 1965, 2023, 1977, 2500, 1976,
       1955, 1968, 1980, 216, 1971, 2060, 214, 1981, 2105, 209, 2104],
      dtype=object)

In [114]:
for index, row in df.iterrows():
    y = int(row['Godina_proizvodnje'])
    if y < 1900 or y > 2023:
        df = df.drop(index)

In [115]:
df['Godina_proizvodnje'].unique()

array([2018, 2011, 2012, 2009, 2016, 1992, 2006, 2007, 2005, 2013, 2019,
       2002, 2001, 2003, 2008, 1996, 2010, 2014, 2004, 1999, 2000, 2017,
       1998, 2015, 1987, 2020, 1994, 1997, 2021, 1995, 2022, 1984, 1990,
       1993, 1991, 1970, 1966, 1988, 1982, 1989, 1986, 1985, 1978, 1983,
       1972, 1965, 2023, 1977, 1976, 1955, 1968, 1980, 1971, 1981],
      dtype=object)

That's it for the cleaning for now. I'm going to save this dataset to a database now, and then use appropriate methods in my analysis to decide if there are any outliers instead of dropping them now based on intuition. I tried here to get rid of None values and obvious mistakes (e.g. I'm confident that none of these cars selling on Index were not made in year 209 or 20). For the rest, I'll do some exploratory analysis first and then decide. 

In [116]:
df.describe()

,Price_in_Euros
count,14038.00
mean,10645.71
std,12812.69
min,1000.00
25%,3200.00
50%,6666.00
75%,13000.00
max,99900.00


### Column Types

Before I save the data to a database I want to get the datatypes right. 

In [117]:
df.dtypes

Active_Since                           object
Times_Viewed                           object
Price_in_Euros                        float64
_Title                                 object
Motor                                  object
Prijeđeni_kilometri                    object
Godina_proizvodnje                     object
Snaga_motora_kW                        object
Boja_vozila                            object
Broj_stupnjeva_na_mjenjaču             object
Broj_vrata                             object
Oblik_karoserije                       object
Prosječna_potrošnja_goriva_l/100km     object
Vlasnik                                object
Vrsta_pogona                           object
Vrsta_mjenjača                         object
Marka                                  object
Model                                  object
Town                                   object
County                                 object
dtype: object

In [118]:
df[['Times_Viewed', 'Prijeđeni_kilometri', 'Snaga_motora_kW', 'Broj_stupnjeva_na_mjenjaču', 'Broj_vrata']] = df[['Times_Viewed', 'Prijeđeni_kilometri', 'Snaga_motora_kW', 'Broj_stupnjeva_na_mjenjaču', 'Broj_vrata']].astype(int)

In [119]:
df['Prosječna_potrošnja_goriva_l/100km'] = df['Prosječna_potrošnja_goriva_l/100km'].astype(float)

In [120]:
df['Active_Since'] = pd.to_datetime(df['Active_Since'], format='%d.%m.%Y')

In [121]:
df['Active_Since'].unique()

array(['2023-03-29T00:00:00.000000000', '2022-12-29T00:00:00.000000000',
       '2023-03-07T00:00:00.000000000', '2023-03-16T00:00:00.000000000',
       '2023-03-08T00:00:00.000000000', '2023-03-28T00:00:00.000000000',
       '2023-03-05T00:00:00.000000000', '2023-01-15T00:00:00.000000000',
       '2022-12-16T00:00:00.000000000', '2023-03-27T00:00:00.000000000',
       '2022-11-15T00:00:00.000000000', '2023-01-16T00:00:00.000000000',
       '2022-10-17T00:00:00.000000000', '2023-03-26T00:00:00.000000000',
       '2020-01-04T00:00:00.000000000', '2022-11-21T00:00:00.000000000',
       '2023-03-25T00:00:00.000000000', '2022-10-27T00:00:00.000000000',
       '2022-07-30T00:00:00.000000000', '2023-01-23T00:00:00.000000000',
       '2023-02-22T00:00:00.000000000', '2022-07-08T00:00:00.000000000',
       '2023-03-24T00:00:00.000000000', '2021-08-12T00:00:00.000000000',
       '2023-03-23T00:00:00.000000000', '2023-02-01T00:00:00.000000000',
       '2023-03-22T00:00:00.000000000', '2022-10-14

In [122]:
df['Godina_proizvodnje'] = pd.to_datetime(df['Godina_proizvodnje'], format='%Y')

In [123]:
df.describe()

,Times_Viewed,Price_in_Euros,Prijeđeni_kilometri,Snaga_motora_kW,Broj_stupnjeva_na_mjenjaču,Broj_vrata,Prosječna_potrošnja_goriva_l/100km
count,14038.00,14038.00,14038.00,14038.00,14038.00,14038.00,4970.00
mean,1135.12,10645.71,204388.44,91.95,5.41,4.69,6.18
std,1318.47,12812.69,92270.75,35.83,0.61,0.74,1.56
min,5.00,1000.00,1.00,12.00,4.00,2.00,0.00
25%,375.00,3200.00,145000.00,70.00,5.00,5.00,5.00
50%,727.00,6666.00,202000.00,85.00,5.00,5.00,6.00
75%,1335.00,13000.00,262000.00,107.00,6.00,5.00,7.00
max,28226.00,99900.00,950000.00,530.00,7.00,6.00,20.00


### Translating

In [124]:
new_names = {
    '_Title': 'Ad_title', 
    'Motor': 'Fuel_Type', 
    'Prijeđeni_kilometri': 'Total_Kilometers',
    'Godina_proizvodnje': 'Production_Year',
    'Snaga_motora_kW': 'Engine_Power_in_kW',
    'Boja_vozila': 'Color',
    'Broj_stupnjeva_na_mjenjaču': 'Transmission_Gears_Count',
    'Broj_vrata': 'Number_of_Doors',
    'Oblik_karoserije': 'Car_Body_Style',
    'Prosječna_potrošnja_goriva_l/100km': 'Fuel_Economy_L/100km',
    'Vlasnik': 'Number_of_Previous_Owners',
    'Vrsta_pogona': 'Drive_Type',
    'Vrsta_mjenjača': 'Transmission_Type',
    'Marka': 'Make'    
}
df.rename(columns=new_names, inplace=True)

In [125]:
df['Number_of_Previous_Owners'].unique()

array(['Drugi', 'Prvi', 'Treći i više'], dtype=object)

In [126]:
for index, row in df.iterrows():
    v = row['Number_of_Previous_Owners']
    nv = None;
    if v == 'Prvi':
        nv = 'One'
    elif v == 'Drugi':
        nv = 'Two'
    else:
        nv = 'Three or more'
    df.at[index, 'Number_of_Previous_Owners'] = nv

In [127]:
df.dtypes

Active_Since                 datetime64[ns]
Times_Viewed                          int32
Price_in_Euros                      float64
Ad_title                             object
Fuel_Type                            object
Total_Kilometers                      int32
Production_Year              datetime64[ns]
Engine_Power_in_kW                    int32
Color                                object
Transmission_Gears_Count              int32
Number_of_Doors                       int32
Car_Body_Style                       object
Fuel_Economy_L/100km                float64
Number_of_Previous_Owners            object
Drive_Type                           object
Transmission_Type                    object
Make                                 object
Model                                object
Town                                 object
County                               object
dtype: object

In [128]:
df['Fuel_Type'].unique()

array(['Diesel', 'Benzin', 'Benzin + LPG', 'Hibridni', 'Električni'],
      dtype=object)

In [129]:
df.loc[df['Fuel_Type'] == 'Benzin', 'Fuel_Type'] = 'Gasoline'
df.loc[df['Fuel_Type'] == 'Benzin + LPG', 'Fuel_Type'] = 'Gasoline + LPG'
df.loc[df['Fuel_Type'] == 'Hibridni', 'Fuel_Type'] = 'Hybrid'
df.loc[df['Fuel_Type'] == 'Električni', 'Fuel_Type'] = 'Electric'

In [130]:
df['Color'].unique()

array(['Crna', 'Bijela', 'Siva', 'Tamno siva', 'Plava', 'Crvena', 'Smeđa',
       'Tamno plava', 'Svijetlo zelena', 'Srebrna', 'Tamno zelena',
       'Bordo crvena', 'Svijetlo siva', 'Svijetlo plava', 'Zlatna',
       'Zelena', 'Bež', 'Žuta', 'Ljubičasta', 'Narančasta', 'Ružičasta'],
      dtype=object)

In [131]:
df.loc[df['Color'] == 'Crna', 'Color'] = 'Black'
df.loc[df['Color'] == 'Bijela', 'Color'] = 'White'
df.loc[df['Color'] == 'Smeđa', 'Color'] = 'Brown'
df.loc[df['Color'] == 'Zlatna', 'Color'] = 'Gold'
df.loc[df['Color'] == 'Srebrna', 'Color'] = 'Silver'
df.loc[df['Color'] == 'Bež', 'Color'] = 'Beige'
df.loc[df['Color'] == 'Žuta', 'Color'] = 'Yellow'
df.loc[df['Color'] == 'Ljubičasta', 'Color'] = 'Purple'
df.loc[df['Color'] == 'Narančasta', 'Color'] = 'Orange'
df.loc[df['Color'] == 'Ružičasta', 'Color'] = 'Pink'
df.loc[df['Color'].isin(['Siva', 'Tamno siva', 'Svijetlo siva']), 'Color'] = 'Grey'
df.loc[df['Color'].isin(['Plava', 'Tamno plava', 'Svijetlo plava']), 'Color'] = 'Blue'
df.loc[df['Color'].isin(['Crvena', 'Bordo crvena']), 'Color'] = 'Red'
df.loc[df['Color'].isin(['Zelena', 'Svijetlo zelena', 'Tamno zelena']), 'Color'] = 'Green'

In [132]:
df['Car_Body_Style'].unique()

array(['Limuzina', 'Karavan', 'Hatchback', 'Kabriolet',
       'Terensko vozilo (SUV)', 'Coupe', 'Kombibus', 'Monovolumen'],
      dtype=object)

In [133]:
df.loc[df['Car_Body_Style'] == 'Limuzina', 'Car_Body_Style'] = 'Sedan'
df.loc[df['Car_Body_Style'] == 'Karavan', 'Car_Body_Style'] = 'Station Wagon'
df.loc[df['Car_Body_Style'] == 'Kabriolet', 'Car_Body_Style'] = 'Convertible'
df.loc[df['Car_Body_Style'] == 'Terensko vozilo (SUV)', 'Car_Body_Style'] = 'SUV'
df.loc[df['Car_Body_Style'] == 'Kombibus', 'Car_Body_Style'] = 'Minivan'
df.loc[df['Car_Body_Style'] == 'Monovolumen', 'Car_Body_Style'] = 'MPV'

In [134]:
df['Drive_Type'].unique()

array(['Prednji', 'Zadnji', '4x4'], dtype=object)

In [135]:
df.loc[df['Drive_Type'] == 'Prednji', 'Drive_Type'] = 'FWD'
df.loc[df['Drive_Type'] == 'Zadnji', 'Drive_Type'] = 'RWD'
df.loc[df['Drive_Type'] == '4x4', 'Drive_Type'] = '4WD'

In [136]:
df['Transmission_Type'].unique()

array(['Mehanički', 'Automatski', 'Automatski-sekvencijski'], dtype=object)

In [137]:
df.loc[df['Transmission_Type'] == 'Mehanički', 'Transmission_Type'] = 'Manual'
df.loc[df['Transmission_Type'] == 'Automatski', 'Transmission_Type'] = 'Automatic'
df.loc[df['Transmission_Type'] == 'Automatski-sekvencijski', 'Transmission_Type'] = 'Semi-Automatic'

In [138]:
df.to_csv('finalData.csv', index=False)

## Uploading to Database

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
POSTGRES_ADDRESS = 'localhost' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = '123456789'
POSTGRES_DBNAME = 'cars'

In [3]:
df = pd.read_csv('data/finalData.csv')

In [4]:
engine = create_engine(f'postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}')

In [5]:
df.to_sql('cars', engine, if_exists='replace')

38

In [6]:
engine.dispose()